<a href="https://colab.research.google.com/github/fishan/Veector/blob/base/Veector_split_DeepSeek_R1_Distill_Qwen_1_5b_int8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === Cell 0: Install Dependencies ===
!pip install numpy psutil torch transformers accelerate bitsandbytes ipfshttpclient qiskit qiskit-aer requests huggingface_hub -q
print("Dependencies installed/checked.")

Dependencies installed/checked.


In [2]:
# === Cell 1: Imports (Corrected and Simplified - FINAL) ===

# --- Standard Imports ---
import numpy as np
import queue
import threading
import time
import random
import psutil
import os
import gc
import pickle
import hashlib
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple, Union
from google.colab import drive, files, userdata # Keep Colab imports
from huggingface_hub import login             # Keep HF import
from transformers import AutoModelForCausalLM, AutoTokenizer # Keep Transformers imports

print("Standard/External imports loaded.")

# --- Optional Imports ---
try:
    import torch
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False
    print("Warning: PyTorch not found. GPU features may be limited.")

try:
    import ipfshttpclient
    IPFS_AVAILABLE = True
except ImportError:
    IPFS_AVAILABLE = False
    # print("Warning: ipfshttpclient not found. IPFS features disabled.")

try:
    from qiskit import QuantumCircuit
    from qiskit.providers.aer import Aer
    from qiskit import execute
    QISKIT_AVAILABLE = True
except ImportError:
    QISKIT_AVAILABLE = False
    # print("Warning: Qiskit not found. Quantum operations disabled.")

print("Optional imports checked.")

# --- Veector Project Imports (Single Correct Block) ---
# Ensure core.py, tensors.py (v0.5.1+), veectordb.py (v0.7.1+),
# operations.py, memory.py are uploaded and accessible.
PROJECT_IMPORTS_OK = False
try:
    # Import core classes/functions needed by THIS script (converter/inference)
    from core import Veector
    from veectordb import VeectorDB # Needed if we re-initialize DB here? Usually not.
    from tensors import (
        TensorCoordinate, create_tensor, # Needed for creating tensors
        # Import ALL necessary TAG and GROUP constants for use in this script
        TAG_CAT_TYPE, TAG_CAT_COMPONENT, TAG_CAT_PRECISION, TAG_CAT_MODEL_FAMILY,
        TAG_CAT_LAYER_IDX, TAG_CAT_FUNCTION, TAG_CAT_DATA_SEMANTIC, TAG_CAT_USER,
        TAG_TYPE_PROCESSOR, TAG_TYPE_KNOWLEDGE, TAG_TYPE_CONVERTER, TAG_TYPE_STATE,
        TAG_COMP_WEIGHTS, TAG_COMP_BIAS, TAG_COMP_EMBEDDING, TAG_COMP_ATTN_Q,
        TAG_COMP_ATTN_K, TAG_COMP_ATTN_V, TAG_COMP_ATTN_O, TAG_COMP_ATTN_QKV,
        TAG_COMP_FFN_GATE, TAG_COMP_FFN_UP, TAG_COMP_FFN_DOWN, TAG_COMP_LAYERNORM,
        TAG_COMP_LM_HEAD, TAG_PREC_FLOAT32, TAG_PREC_FLOAT16, TAG_PREC_BFLOAT16,
        TAG_PREC_INT8, TAG_PREC_INT4, TAG_MODEL_QWEN2, TAG_MODEL_LLAMA3,
        TAG_MODEL_DEEPSEEK, TAG_FUNC_LINEAR, TAG_FUNC_ATTENTION, TAG_FUNC_FFN,
        TAG_FUNC_EMBED_LOOKUP, TAG_FUNC_CAST_DTYPE, TAG_FUNC_RESHAPE,
        TAG_SEMANTIC_HIDDEN_STATE, TAG_SEMANTIC_LOGITS, TAG_SEMANTIC_TOKEN_IDS,
        TAG_SEMANTIC_KV_CACHE, tag_layer,
        GROUP_IDX_QWEN_KNOWLEDGE, GROUP_IDX_QWEN_PROCESSOR
    )
    # Only import from operations/memory if DIRECTLY used in THIS script, otherwise core.py handles it
    # from operations import * # Generally not needed here
    # from memory import Memory # Generally not needed here

    print("Veector project components imported successfully for this script.")
    PROJECT_IMPORTS_OK = True

except ImportError as e:
    print(f"---!!! FATAL ERROR (ImportError) !!! ---")
    print(f"Specific error: {e}")
    print(f"Could not import required name from core.py or tensors.py.")
    print(f"Ensure files are UP-TO-DATE (tensors v0.5.1+, core v0.5.2+), CORRECT, and ACCESSIBLE.")
    print(f"-----------------------------------------")
    # Optionally define dummies if needed for notebook structure
except Exception as other_e:
    print(f"---!!! FATAL ERROR (Other Exception during Import) !!! ---")
    print(f"Specific error: {other_e}")
    import traceback
    traceback.print_exc()
    print(f"Check imported files for syntax errors.")
    print(f"----------------------------------------------------------")

# Removed the redundant import check block ('Checking imports...')

Standard/External imports loaded.
Optional imports checked.
  Imported VeectorDB (v0.9.7)
  Imported tensors (v0.7.6)
  Imported operations (v0.8.3)
  Imported Memory (v0.1.0)
Core components imported successfully.
---!!! FATAL ERROR (ImportError) !!! ---
Specific error: cannot import name 'TAG_CAT_TYPE' from 'tensors' (/content/tensors.py)
Could not import required name from core.py or tensors.py.
Ensure files are UP-TO-DATE (tensors v0.5.1+, core v0.5.2+), CORRECT, and ACCESSIBLE.
-----------------------------------------


In [3]:
# Очистка директории для чистоты эксперимента
!rm -rf data/
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)


In [4]:
# --- Configuration ---

# Аутентификация с Hugging Face
hf_token = userdata.get('HF_TOKEN')
if not hf_token:
    raise ValueError("Добавь HF_TOKEN в секреты Colab!")
login(hf_token)
print("Аутентификация прошла успешно")

# Подключение Google Drive
drive.mount('/content/drive')
print("Google Drive подключён")

HF_MODEL_NAME = "DeepSeek-R1-Distill-Qwen-1.5B"
# Определяем ОДИН основной путь к БД (например, в data/db/)
DB_PATH = Path("./data/db/")
DB_PATH.mkdir(parents=True, exist_ok=True) # Создаем data/db, если ее нет
print(f"Using Main Veector DB Path: {DB_PATH.resolve()}")

# Set data type (bfloat16 might not be fully supported everywhere, float16 is safer)
TORCH_DTYPE = torch.float16 # Use float16 for wider compatibility

print(f"Model to convert: {HF_MODEL_NAME}")
print(f"Target Veector DB: {DB_PATH}")
print(f"Target dtype: {TORCH_DTYPE}")

Аутентификация прошла успешно
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive подключён
Using Main Veector DB Path: /content/data/db
Model to convert: DeepSeek-R1-Distill-Qwen-1.5B
Target Veector DB: data/db
Target dtype: torch.float16


In [5]:
# === Cell 2: Tag Ontology and Mappings Definition (Sync with tensors.py v0.7.0) ===

import torch # Ensure torch is imported for dtype checking if needed later
import numpy as np # Ensure numpy is imported
from typing import Dict, List, Any, Optional, Tuple, Union # Import typing for hints

# --- Version (for tracking changes in this cell) ---
CONVERTER_CELL2_VERSION = "Synced with tensors.py v0.7.0"
print(f"--- Running Converter Cell 2 v{CONVERTER_CELL2_VERSION} ---")

# --- Type Hint for Metadata Tuple (from tensors.py) ---
# Needed if any functions within Colab cells might use this type hint
MetadataTuple = Tuple[
    List[Union[float, int]],         # [0] data_description
    List[int],                       # [1] coord
    List[int],                       # [2] shape
    List[int],                       # [3] tags
    Optional[Dict],                  # [4] ops_sequences
    Optional[Dict],                  # [5] interface
    Optional[List],                  # [6] filters
    Optional[List],                  # [7] exit_gates
    List[int],                       # [8] lifecycle
    Optional[List[str]]              # [9] parents
]

# --- Simplified Tag Ontology (Flat Integers with Ranges - from tensors.py v0.7.0) ---
# 1-9: Tensor Type
TAG_TYPE_PROCESSOR = 1
TAG_TYPE_KNOWLEDGE = 2
TAG_TYPE_CONVERTER = 3
TAG_TYPE_STATE = 4
# 10-19: Model Family
TAG_MODEL_QWEN2 = 10
TAG_MODEL_LLAMA3 = 11
TAG_MODEL_DEEPSEEK = 12
# 20-29: Precision
TAG_PREC_FLOAT32 = 20
TAG_PREC_FLOAT16 = 21
TAG_PREC_BFLOAT16 = 22
TAG_PREC_INT8 = 23
TAG_PREC_INT4 = 24
# 30-49: Component Type
TAG_COMP_WEIGHTS = 30
TAG_COMP_BIAS = 31
TAG_COMP_EMBEDDING = 32
TAG_COMP_ATTN_Q = 33
TAG_COMP_ATTN_K = 34
TAG_COMP_ATTN_V = 35
TAG_COMP_ATTN_O = 36
TAG_COMP_ATTN_QKV = 37
TAG_COMP_FFN_GATE = 38
TAG_COMP_FFN_UP = 39
TAG_COMP_FFN_DOWN = 40
TAG_COMP_LAYERNORM = 41
TAG_COMP_LM_HEAD = 42
# 50-59: Function
TAG_FUNC_LINEAR = 50
TAG_FUNC_ATTENTION = 51
TAG_FUNC_FFN = 52
TAG_FUNC_EMBED_LOOKUP = 53
TAG_FUNC_CAST_DTYPE = 54
TAG_FUNC_RESHAPE = 55
# 60-69: Data Semantic Type
TAG_SEMANTIC_HIDDEN_STATE = 60
TAG_SEMANTIC_LOGITS = 61
TAG_SEMANTIC_TOKEN_IDS = 62
TAG_SEMANTIC_KV_CACHE = 63
# 100-999: Layer Index
LAYER_IDX_TAG_OFFSET = 100

def tag_layer(idx: int) -> int:
    """Generates a layer tag using an offset."""
    if not isinstance(idx, int): raise TypeError(f"Layer index must be an integer, got {type(idx)}")
    if idx < 0: raise ValueError(f"Invalid layer index for tagging: {idx}. Must be non-negative.")
    return LAYER_IDX_TAG_OFFSET + idx
# 1000+: User Defined Tags
USER_TAG_OFFSET = 1000
# --- End of Tags ---
print("Simplified tag ontology (flat integers) defined.")

# --- Group ID Constants (from tensors.py v0.7.0) ---
GROUP_IDX_QWEN_KNOWLEDGE = 100
GROUP_IDX_QWEN_PROCESSOR = 500
GROUP_IDX_LLAMA_KNOWLEDGE = 101
GROUP_IDX_LLAMA_PROCESSOR = 501
GROUP_IDX_DEEPSEEK_KNOWLEDGE = 102 # Added constant
# GROUP_IDX_DEEPSEEK_PROCESSOR = 502 # Optional
GROUP_IDX_GENERIC_PROCESSOR = 50
print(f"Group Indices defined: QwenK={GROUP_IDX_QWEN_KNOWLEDGE}, QwenP={GROUP_IDX_QWEN_PROCESSOR}, DeepSeekK={GROUP_IDX_DEEPSEEK_KNOWLEDGE}")


# --- Mappings (from tensors.py v0.7.0) ---
# 1. DATA_TYPE_MAPPING
DATA_TYPE_MAPPING = {
    "knowledge": 1,
    "processor": 2,
    "converter": 3,
    "state": 4,
}
REVERSE_DATA_TYPE_MAPPING = {
    1: "knowledge",
    2: "processor",
    3: "converter",
    4: "state",
}
print(f"DATA_TYPE_MAPPING defined: {DATA_TYPE_MAPPING}")

# 2. DTYPE_MAPPING
DTYPE_MAPPING = {
    # Standard Names
    'float32': 1, 'float16': 2, 'bfloat16': 3, 'int8': 4, 'int4': 5,
    'int32': 6, 'int64': 7, 'bool': 8, 'complex64': 9, 'complex128': 10,
    # Numpy Types
    np.float32: 1, np.float16: 2, np.int8: 4, np.int32: 6, np.int64: 7,
    np.bool_: 8, np.complex64: 9, np.complex128: 10,
    # PyTorch Types (as strings and potentially objects if torch loaded)
    'torch.float32': 1, 'torch.float16': 2, 'torch.bfloat16': 3, 'torch.int8': 4,
    'torch.int32': 6, 'torch.int64': 7, 'torch.bool': 8,
    'torch.complex64': 9, 'torch.complex128': 10,
}
# Add torch objects if torch is available
if 'torch' in globals():
    DTYPE_MAPPING[torch.float32] = 1
    DTYPE_MAPPING[torch.float16] = 2
    DTYPE_MAPPING[torch.bfloat16] = 3
    DTYPE_MAPPING[torch.int8] = 4
    DTYPE_MAPPING[torch.int32] = 6
    DTYPE_MAPPING[torch.int64] = 7
    DTYPE_MAPPING[torch.bool] = 8
    DTYPE_MAPPING[torch.complex64] = 9
    DTYPE_MAPPING[torch.complex128] = 10

REVERSE_DTYPE_MAPPING = {
    1: 'float32', 2: 'float16', 3: 'bfloat16', 4: 'int8', 5: 'int4',
    6: 'int32', 7: 'int64', 8: 'bool', 9: 'complex64', 10: 'complex128',
}
print(f"DTYPE_MAPPING defined.")

# 3. STATUS_MAPPING
STATUS_MAPPING = {
    "active": 1,
    "archived": 0
}
REVERSE_STATUS_MAPPING = {
    1: "active",
    0: "archived"
}
print(f"STATUS_MAPPING defined: {STATUS_MAPPING}")

# --- Metadata Encoding Configuration (from tensors.py v0.7.0) ---
METADATA_STRUCTURE_VERSION = 1.1
print(f"Metadata Structure Version: {METADATA_STRUCTURE_VERSION}")

print("Tag ontology, Group IDs, Mappings, and Config defined for Cell 2.")

--- Running Converter Cell 2 vSynced with tensors.py v0.7.0 ---
Simplified tag ontology (flat integers) defined.
Group Indices defined: QwenK=100, QwenP=500, DeepSeekK=102
DATA_TYPE_MAPPING defined: {'knowledge': 1, 'processor': 2, 'converter': 3, 'state': 4}
DTYPE_MAPPING defined.
STATUS_MAPPING defined: {'active': 1, 'archived': 0}
Metadata Structure Version: 1.1
Tag ontology, Group IDs, Mappings, and Config defined for Cell 2.


In [16]:
!ls -lh *.py

-rw-r--r-- 1 root root  67K Apr  4 08:06 core.py
-rw-r--r-- 1 root root 2.9K Apr  4 04:43 memory.py
-rw-r--r-- 1 root root  22K Apr  4 07:45 operations.py
-rw-r--r-- 1 root root  23K Apr  4 07:49 tensors.py
-rw-r--r-- 1 root root  39K Apr  4 07:47 veectordb.py


In [6]:
# === Cell 3: Initialize Veector (SINGLE Instance) ===
from core import Veector # Импортируем класс Veector из core.py
try:
    # Используем этот путь при инициализации
    vec = Veector(db_dir=DB_PATH, ipfs_enabled=False)
    print(f"Veector core initialized using DB at: {DB_PATH.resolve()}")
except Exception as e:
    print(f"FATAL: Veector initialization failed: {e}")
    raise RuntimeError("Veector Core failed to initialize") from e

--- Initializing Veector Core v0.7.2 ---
    Requires: tensors v0.7.6+, veectordb v0.9.7+, operations v0.8.3+
    IPFS: False, Address: /ip4/127.0.0.1/tcp/5001
--- Initializing VeectorDB v0.9.7 (requires tensors v0.7.6+) ---
VeectorDB v0.9.7 initialized at /content/data/db. Index entries: 0.
VeectorDB initialized.
Cache initialized: Size=1000, Strategy=LRU
Initialized 74 core operations.
Veector core initialized using DB at: /content/data/db


In [7]:
# === Cell 4: Load Hugging Face Model ===

model = None
tokenizer = None
try:
    model = AutoModelForCausalLM.from_pretrained(f"deepseek-ai/{HF_MODEL_NAME}", torch_dtype=TORCH_DTYPE, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(f"deepseek-ai/{HF_MODEL_NAME}", trust_remote_code=True)
    model.eval() # Set to evaluation mode
    print(f"Successfully loaded HF model: {HF_MODEL_NAME}")
    print(f"Model config: {model.config}")
except Exception as e:
    print(f"FATAL: Failed to load HF model '{HF_MODEL_NAME}': {e}")
    # Stop execution
    raise RuntimeError(f"Hugging Face model loading failed") from e

# Clean up GPU memory if possible after loading
if TORCH_AVAILABLE and torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()
print("Model loaded and memory potentially cleaned.")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Successfully loaded HF model: DeepSeek-R1-Distill-Qwen-1.5B
Model config: Qwen2Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 131072,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.50.3",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}

Model loaded and memory potentially cleaned.


Test inference

In [8]:
# Ubedis', chto peremennye 'model' i 'tokenizer' zagruzheny iz predydushhih jacheek
# model = AutoModelForCausalLM.from_pretrained(...)
# tokenizer = AutoTokenizer.from_pretrained(...)
if 'model' not in locals() or 'tokenizer' not in locals():
    raise NameError("Model or tokenizer not loaded")
model.eval() # Perevodim model' v rezhim inference
# Esli est' GPU, perenosim model' tuda
if torch.cuda.is_available():
    model.to('cuda')

In [9]:
prompt = "Hello!"
messages = [{"role": "user", "content": prompt}]
# Ispol'zuem standartnyj metod dlja korrektnogo formatirovanija
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # Dobavljaet markery nachala otveta assistenta
    tokenize=True,
    return_tensors="pt" # Vozvrashhaem PyTorch tenzory
)
# Perenosim na GPU, esli ispol'zuem
if torch.cuda.is_available():
    input_ids = input_ids.to('cuda')

In [13]:
# Parametry generacii (mozhno poigrat'sja)
max_new_tokens = 500
temperature = 0.6
top_p = 0.95
eos_token_id = tokenizer.eos_token_id # Berem EOS ID iz tokenizatora

print(f"\n--- Starting generation with model.generate() ---")
print(f"Prompt: '{prompt}'")
print(f"Parameters: temp={temperature}, top_p={top_p}, max_new={max_new_tokens}")

# Sam process generacii
# model.generate ispol'zuet KV-cache i optimizacii PyTorch
generated_ids = model.generate(
    input_ids,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=True, # Vkljuchaem semplirovanie
    pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else eos_token_id # Vazhno dlja nekotoryh modelej
    # eos_token_id=eos_token_id # Mozhno javno ukazat', no obychno on beretsja iz konfiga
)
print("--- Generation finished ---")


--- Starting generation with model.generate() ---
Prompt: 'Hello!'
Parameters: temp=0.6, top_p=0.95, max_new=500
--- Generation finished ---


In [14]:
# Dekodiruem tol'ko sgenerirovannuju chast'
# generated_ids[0] soderzhit ID vhodnogo prompta + sgenerirovannye ID
output_ids = generated_ids[0][input_ids.shape[1]:]
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)

print("\n--- Generated Text (Original Model) ---")
print(generated_text)


--- Generated Text (Original Model) ---
Alright, the user just said "Hello!" to me. I should respond in a friendly and approachable way. Maybe I can greet them back and ask how I can assist. Keeping it simple and open-ended should encourage them to share what they need help with.
</think>

Hello! How can I assist you today?


Converting knowledge tensors

In [8]:
# === Cell 5: Convert Parameters to Knowledge Tensors (Transposed Weights) ===

import gc
import pickle
import time
import traceback
from pathlib import Path

import numpy as np
import torch

# --- Импорты из проекта (убедись, что версии >= 0.6.12 и >= 0.7.6) ---
try:
    from tensors import (
        TENSORS_VERSION, TensorCoordinate, create_tensor, MetadataTuple,
        validate_tensor_tuple, validate_tensor, DTYPE_MAPPING,
        TAG_TYPE_KNOWLEDGE, TAG_MODEL_DEEPSEEK, TAG_COMP_WEIGHTS, TAG_COMP_BIAS,
        TAG_COMP_EMBEDDING, TAG_COMP_LM_HEAD, TAG_COMP_LAYERNORM, TAG_COMP_ATTN_Q,
        TAG_COMP_ATTN_K, TAG_COMP_ATTN_V, TAG_COMP_ATTN_O, TAG_COMP_FFN_GATE,
        TAG_COMP_FFN_UP, TAG_COMP_FFN_DOWN, tag_layer, GROUP_IDX_QWEN_KNOWLEDGE,
        TAG_PREC_FLOAT32, TAG_PREC_FLOAT16, TAG_PREC_BFLOAT16, TAG_PREC_INT8
    )
    if TENSORS_VERSION < "0.7.6":
        raise ImportError(f"Requires tensors v0.7.6+, found v{TENSORS_VERSION}")
    from core import Veector, CORE_VERSION
    if CORE_VERSION < "0.6.12":
        raise ImportError(f"Requires core v0.6.12+, found v{CORE_VERSION}")
except ImportError as e:
    print(f"FATAL ERROR: Import failed: {e}")
    raise

# --- Версия Ячейки ---
CONVERTER_CELL5_VERSION = "Hybrid v0.7.6 + Quant + Transpose v2"
# --- Конец Версии ---

print(f"--- Running Converter Cell 5 v{CONVERTER_CELL5_VERSION} ---")
start_cell5_time = time.time()

# --- Проверка необходимых переменных ---
if 'vec' not in locals() or vec is None:
    raise NameError("'vec' object not defined.")
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
    raise NameError("DB_PATH not defined or invalid.")
if 'model' not in locals() or model is None:
    raise NameError("HF 'model' not loaded.")
if 'HF_MODEL_NAME' not in locals() or not HF_MODEL_NAME:
    raise NameError("HF_MODEL_NAME not defined.")

# --- Переинициализация DB (если необходимо) ---
if not hasattr(vec, 'db') or vec.db is None:
    try:
        print("Attempting DB re-init for Cell 5...")
        # Импортируем только если нужно, чтобы избежать ненужных импортов вверху
        from veectordb import VeectorDB
        vec.db = VeectorDB(db_dir=DB_PATH)
        print("DB connection re-established.")
    except Exception as db_reinit_e:
        raise AttributeError(f"DB re-init failed: {db_reinit_e}")
else:
    print("'vec' object found and DB connection seems active.")

# --- Инициализация ---
ORIGINAL_NAME_TO_ID_MAP: Dict[str, int] = {}
ID_TO_ORIGINAL_NAME_MAP: Dict[int, str] = {}
NEXT_NAME_ID: int = 0
print("Initialized Name <-> ID mapping dictionaries.")

knowledge_map: Dict[str, str] = {} # Карта Имя -> ID Знания
param_count: int = 0
conversion_errors: int = 0

# --- Вспомогательная функция для ID ---
def get_or_create_name_id(name: Optional[str]) -> int:
    """Assigns and returns a unique ID for a parameter name."""
    global NEXT_NAME_ID, ORIGINAL_NAME_TO_ID_MAP, ID_TO_ORIGINAL_NAME_MAP
    if not name:
        return -1
    if name in ORIGINAL_NAME_TO_ID_MAP:
        return ORIGINAL_NAME_TO_ID_MAP[name]
    current_id = NEXT_NAME_ID
    ORIGINAL_NAME_TO_ID_MAP[name] = current_id
    ID_TO_ORIGINAL_NAME_MAP[current_id] = name
    NEXT_NAME_ID += 1
    return current_id

# --- Параметры конвертации ---
default_precision_tag = TAG_PREC_FLOAT16
default_torch_dtype = torch.float16
if 'TORCH_DTYPE' in locals(): # Определено в Cell 1
    default_torch_dtype = TORCH_DTYPE
    if TORCH_DTYPE == torch.float16: default_precision_tag = TAG_PREC_FLOAT16
    elif TORCH_DTYPE == torch.bfloat16: default_precision_tag = TAG_PREC_BFLOAT16
    elif TORCH_DTYPE == torch.float32: default_precision_tag = TAG_PREC_FLOAT32
    elif TORCH_DTYPE == torch.int8: default_precision_tag = TAG_PREC_INT8

knowledge_group_idx = GROUP_IDX_QWEN_KNOWLEDGE # 100
model_tag = TAG_MODEL_DEEPSEEK # 12

print(f"\n--- Creating Knowledge Tensors (Group: {knowledge_group_idx}) ---")
print(f"    Model Tag: {model_tag}")
print(f"    Default Precision Tag: {default_precision_tag}")
print(f"    Quantizing Embed/LMHead to INT8. Transposing Linear Weights.")

# --- Основной цикл конвертации ---
total_params = sum(1 for _ in model.named_parameters())
print(f"Found {total_params} parameters to process.")

for idx, (name, param) in enumerate(model.named_parameters()):
    loop_start_time = time.time()
    print(f"\nProcessing Param {idx+1}/{total_params}: {name}")
    print(f"  Original Shape: {param.shape} | Dtype: {param.dtype}")

    # Инициализация переменных цикла
    param_data_fp32: Optional[np.ndarray] = None
    knowledge_data_to_pass: Optional[np.ndarray] = None
    tags: List[int] = []
    metadata_extra_to_pass: Optional[Dict] = None
    dtype_to_pass: Any = None
    final_tags: List[int] = []
    knowledge_coord: Optional[TensorCoordinate] = None
    name_id: int = -1
    create_result: Optional[List] = None
    knowledge_id: Optional[str] = None
    requires_transpose: bool = False

    try:
        # Шаг 1-3: Получение данных, ID, Тегов, Координат
        param_data_fp32 = param.data.cpu().to(torch.float32).numpy()
        name_id = get_or_create_name_id(name)
        tags = [TAG_TYPE_KNOWLEDGE, model_tag]
        layer_idx = -1
        group_idx = knowledge_group_idx
        coord_x = 0
        current_nest = 1 # По умолчанию Nest=1 для знаний
        is_weight = name.endswith(".weight")
        is_bias = name.endswith(".bias")

        if is_weight: tags.append(TAG_COMP_WEIGHTS)
        elif is_bias: tags.append(TAG_COMP_BIAS)

        # Определение компонента, X координа и флага транспонирования
        if "model.embed_tokens.weight" in name:
             tags.append(TAG_COMP_EMBEDDING); coord_x = 0
        elif "lm_head.weight" in name:
             tags.append(TAG_COMP_LM_HEAD); coord_x = 1; requires_transpose = True
        elif "model.norm.weight" in name:
             layer_idx = model.config.num_hidden_layers; tags.append(TAG_COMP_LAYERNORM); coord_x = 0
        elif ".layers." in name:
            try:
                layer_part = name.split('.layers.')[1]
                layer_idx = int(layer_part.split('.')[0])
                if layer_idx >= 0: tags.append(tag_layer(layer_idx))
                else: raise ValueError(f"Invalid L idx: {layer_idx}")

                component_tag_layer = None
                if "self_attn" in name:
                    if "q_proj.weight" in name: component_tag_layer = TAG_COMP_ATTN_Q; coord_x = 10; requires_transpose = True
                    elif "q_proj.bias" in name: component_tag_layer = TAG_COMP_ATTN_Q; coord_x = 11
                    elif "k_proj.weight" in name: component_tag_layer = TAG_COMP_ATTN_K; coord_x = 20; requires_transpose = True
                    elif "k_proj.bias" in name: component_tag_layer = TAG_COMP_ATTN_K; coord_x = 21
                    elif "v_proj.weight" in name: component_tag_layer = TAG_COMP_ATTN_V; coord_x = 30; requires_transpose = True
                    elif "v_proj.bias" in name: component_tag_layer = TAG_COMP_ATTN_V; coord_x = 31
                    elif "o_proj.weight" in name: component_tag_layer = TAG_COMP_ATTN_O; coord_x = 40; requires_transpose = True
                elif "mlp" in name:
                    if "gate_proj.weight" in name: component_tag_layer = TAG_COMP_FFN_GATE; coord_x = 50; requires_transpose = True
                    elif "up_proj.weight" in name: component_tag_layer = TAG_COMP_FFN_UP; coord_x = 60; requires_transpose = True
                    elif "down_proj.weight" in name: component_tag_layer = TAG_COMP_FFN_DOWN; coord_x = 70; requires_transpose = True
                elif "input_layernorm.weight" in name: component_tag_layer = TAG_COMP_LAYERNORM; coord_x = 1
                elif "post_attention_layernorm.weight" in name: component_tag_layer = TAG_COMP_LAYERNORM; coord_x = 2

                if component_tag_layer: tags.append(component_tag_layer)
                elif not is_weight and not is_bias: print(f"  WARN: Unrecognized comp in L{layer_idx}: {name}"); coord_x = 99
            except Exception as parse_e:
                print(f"  Error parsing layer for {name}: {parse_e}"); conversion_errors += 1; continue
        else:
            print(f"  WARN: Param unmatched: {name}"); layer_idx = -1; coord_x = 999

        knowledge_coord = TensorCoordinate(layer=layer_idx, group=group_idx, nest=current_nest, x=coord_x)

        # Шаг 4: Квантование / Приведение типов / Транспонирование
        quantization_scale = None
        current_precision_tag = default_precision_tag
        data_before_save = None

        if name == "model.embed_tokens.weight" or name == "lm_head.weight":
            if np.issubdtype(param_data_fp32.dtype, np.floating):
                try:
                    abs_max = np.max(np.abs(param_data_fp32)); scale = 1.0
                    if abs_max >= 1e-9: scale = abs_max / 127.0
                    scale = max(scale, 1e-9) # Prevent division by zero
                    quantized_data = np.round(param_data_fp32 / scale).astype(np.int8)
                    data_before_save = quantized_data; dtype_to_pass = np.int8
                    quantization_scale = float(scale); current_precision_tag = TAG_PREC_INT8
                    metadata_extra_to_pass = {"quantization_scale": quantization_scale}
                    # Транспонируем только LM Head ПОСЛЕ квантования
                    if name == "lm_head.weight": # requires_transpose is True here
                        print("  Transposing quantized LM Head weights...")
                        data_before_save = data_before_save.T
                except Exception as quant_e:
                     print(f"  ERROR quantizing {name}: {quant_e}"); conversion_errors += 1; continue
            else: # Не float - не квантуем
                 data_before_save = param_data_fp32; dtype_to_pass = data_before_save.dtype; current_precision_tag = DTYPE_MAPPING.get(dtype_to_pass, default_precision_tag); metadata_extra_to_pass = None
                 if requires_transpose: # Все равно транспонируем, если нужно
                      print(f"  Transposing non-quantized {name}...")
                      data_before_save = data_before_save.T
        else: # Не embedding и не lm_head
            try:
                target_np_dtype = default_torch_dtype.numpy_dtype if hasattr(default_torch_dtype, 'numpy_dtype') else np.float16
                data_before_save = param_data_fp32.astype(target_np_dtype)
                dtype_to_pass = data_before_save.dtype; current_precision_tag = default_precision_tag
                metadata_extra_to_pass = None
                # Транспонируем если нужно
                if requires_transpose:
                    print(f"  Transposing {name} weights...")
                    data_before_save = data_before_save.T
            except Exception as cast_e:
                 print(f"  ERROR casting/transposing {name}: {cast_e}"); conversion_errors += 1; continue

        # Финальные данные для сохранения
        knowledge_data_to_pass = data_before_save
        final_shape_to_save = knowledge_data_to_pass.shape if knowledge_data_to_pass is not None else None

        # Шаг 5: Финализация тегов
        final_tags = list(tags)
        if current_precision_tag != default_precision_tag and default_precision_tag in final_tags:
            final_tags.remove(default_precision_tag)
        if current_precision_tag:
            final_tags.append(current_precision_tag)
        final_tags = sorted(list(set(final_tags)))

        print(f"  Final Tags: {final_tags}"); print(f"  Coordinate: {knowledge_coord}")
        print(f"  Data to save: dtype={dtype_to_pass}, shape={final_shape_to_save}") # Используем final_shape_to_save
        if metadata_extra_to_pass: print(f"  Extra Metadata: {metadata_extra_to_pass}")

        # Шаг 6: Создание Тензора
        create_result = vec.create_tensor(
             coord=knowledge_coord,
             tensor_type="knowledge",
             knowledge_data=knowledge_data_to_pass, # Передаем возможно транспонированные данные
             tags=final_tags,
             dtype=dtype_to_pass,
             shape=final_shape_to_save, # Передаем правильную форму
             name_id=name_id,
             metadata_extra=metadata_extra_to_pass,
             status="active"
         )

        # Шаг 8: Сохранение Тензора
        knowledge_id = vec.save_tensor(create_result) # Передаем список

        if knowledge_id:
            knowledge_map[name] = knowledge_id
            param_count += 1
        else:
            conversion_errors += 1
            print(f"  ERROR saving tensor for {name}")

    except Exception as create_save_e:
        print(f"  ERROR during create/save for {name}: {create_save_e}")
        traceback.print_exc(); conversion_errors += 1
    finally:
        if param_data_fp32 is not None:
            del param_data_fp32 # Освобождаем память
        loop_end_time = time.time()
        # print(f"  Param {idx+1} time: {loop_end_time - loop_start_time:.2f}s") # Сократим лог

# --- Конец Цикла ---

print(f"\n--- Finished saving {param_count} knowledge tensors to {vec.db.db_root_path if vec.db else 'N/A'} ---")
if conversion_errors > 0:
    print(f"!!! WARNING: {conversion_errors} errors occurred during knowledge conversion !!!")

# --- Сохранение Name ID Map ---
name_map_file = DB_PATH / f"{HF_MODEL_NAME}_name_id_map.pkl"
try:
    map_data_to_save = {
        "name_to_id": ORIGINAL_NAME_TO_ID_MAP,
        "id_to_name": ID_TO_ORIGINAL_NAME_MAP,
        "next_id": NEXT_NAME_ID
    }
    with open(name_map_file, 'wb') as f:
        pickle.dump(map_data_to_save, f)
    print(f"\nName <-> ID map saved to {name_map_file}")
except Exception as e:
    print(f"  Error saving name ID map: {e}")

# --- Сохранение Knowledge Map (для Cell 5.5) ---
# Имя файла определяется в Cell 4.5, но мы его здесь переопределим для надежности
knowledge_map_filename = f"{HF_MODEL_NAME}_knowledge_map.pkl"
knowledge_map_filepath = DB_PATH / knowledge_map_filename
try:
    print(f"\n--- Saving Knowledge Map (for Cell 5.5) ---")
    with open(knowledge_map_filepath, 'wb') as f:
        pickle.dump(knowledge_map, f)
    print(f"  Knowledge map saved to {knowledge_map_filepath}")
except Exception as e:
    print(f"  Error saving knowledge map: {e}")
    # Важно: если карта не сохранилась, Cell 6 не сможет загрузить ее позже
    # Можно добавить обработку этой ошибки, если нужно
    conversion_errors += 1 # Считаем это ошибкой конвертации

print(f"\n'knowledge_map' created with {len(knowledge_map)} entries for Cell 5.5.")

# --- Очистка ---
# (Без изменений)
if 'torch' in locals() and hasattr(torch, 'cuda') and torch.cuda.is_available():
     torch.cuda.empty_cache()
gc.collect()
print("\nMemory cleanup attempted.")
print("DB connection remains open for Cell 5.5/6.")

# --- Завершение Ячейки 5 ---
end_cell5_time = time.time()
print(f"--- Cell 5 Finished in {end_cell5_time - start_cell5_time:.2f} seconds ---")

--- Running Converter Cell 5 vHybrid v0.7.6 + Quant + Transpose v2 ---
'vec' object found and DB connection seems active.
Initialized Name <-> ID mapping dictionaries.

--- Creating Knowledge Tensors (Group: 100) ---
    Model Tag: 12
    Default Precision Tag: 21
    Quantizing Embed/LMHead to INT8. Transposing Linear Weights.
Found 339 parameters to process.

Processing Param 1/339: model.embed_tokens.weight
  Original Shape: torch.Size([151936, 1536]) | Dtype: torch.float16
  Final Tags: [2, 12, 23, 30, 32]
  Coordinate: L-1_G100_N1_X0_Y0_Z0
  Data to save: dtype=<class 'numpy.int8'>, shape=(151936, 1536)
  Extra Metadata: {'quantization_scale': 0.0025990402791649103}
DEBUG INDEX UPDATE: Adding/Updating ID fb4ef375e208da470d6841a89c441ca29e016873a80c8519660c22cfa227be8d -> Type: knowledge, Status: active, Coords: L-1_G100_N1_X0_Y0_Z0

Processing Param 2/339: model.layers.0.self_attn.q_proj.weight
  Original Shape: torch.Size([1536, 1536]) | Dtype: torch.float16
  Transposing model.l

In [9]:
# === Cell 5.5: Save Intermediate Data for Cell 6 ===

import pickle
from pathlib import Path
import os

print("\n--- Running Cell 5.5: Saving Intermediate Data ---")

# --- Проверка наличия необходимых переменных из предыдущих ячеек ---
if 'knowledge_map' not in locals() or not isinstance(knowledge_map, dict):
    raise NameError("Variable 'knowledge_map' not found or invalid. Ensure Cell 5 ran successfully.")
if 'model' not in locals() or model is None:
    # Нам нужен как минимум конфиг модели для num_layers
    raise NameError("Variable 'model' (or model.config) not found. Ensure Cell 4 ran successfully.")
if 'HF_MODEL_NAME' not in locals() or not HF_MODEL_NAME:
     raise NameError("Variable 'HF_MODEL_NAME' not defined. Check Cell 1.")
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
     raise NameError("Variable 'DB_PATH' not defined or invalid. Check Cell 1.")

# --- Данные для сохранения ---
# Сохраняем только конфиг, а не всю модель, для экономии места
cell6_input_data = {
    'knowledge_map': knowledge_map,
    'model_config': model.config, # Сохраняем конфиг
    'hf_model_name': HF_MODEL_NAME,
    'db_path': str(DB_PATH.resolve()) # Сохраняем путь к БД как строку
}

# --- Имя файла и сохранение ---
intermediate_filename = f"{HF_MODEL_NAME}_cell6_input_data.pkl"
intermediate_filepath = DB_PATH / intermediate_filename

try:
    # Убедимся, что директория DB_PATH существует
    DB_PATH.mkdir(parents=True, exist_ok=True)

    print(f"Saving intermediate data to: {intermediate_filepath}")
    with open(intermediate_filepath, 'wb') as f:
        pickle.dump(cell6_input_data, f, pickle.HIGHEST_PROTOCOL)
    print("Intermediate data saved successfully.")
    print(f"  Knowledge map entries: {len(knowledge_map)}")
    print(f"  Model Config Type: {type(model.config)}")

except Exception as e:
    print(f"---!!! ERROR saving intermediate data: {e} !!!---")
    # Можно добавить raise e, если критично прервать выполнение
else:
    print("--- Cell 5.5 Finished ---")

# --- Очистка памяти от модели (если она больше не нужна до перезапуска) ---
# Раскомментируй, если хочешь освободить память после сохранения промежуточных данных
# import gc
# if 'model' in locals(): del model
# if 'torch' in locals() and hasattr(torch, 'cuda') and torch.cuda.is_available(): torch.cuda.empty_cache()
# gc.collect()
# print("Cleaned up model from memory (optional).")


--- Running Cell 5.5: Saving Intermediate Data ---
Saving intermediate data to: data/db/DeepSeek-R1-Distill-Qwen-1.5B_cell6_input_data.pkl
Intermediate data saved successfully.
  Knowledge map entries: 339
  Model Config Type: <class 'transformers.models.qwen2.configuration_qwen2.Qwen2Config'>
--- Cell 5.5 Finished ---


In [10]:
# === Cell 5.6: Verify Tensor Data ===
import numpy as np
import torch
import traceback

print("\n--- Running Cell 5.6: Verify Tensor Data ---")

# --- Proverka nalichija peremennyh ---
if 'model' not in locals() or model is None: raise NameError("HF 'model' not loaded.")
if 'vec' not in locals() or vec is None: raise NameError("'vec' object not defined.")
if 'knowledge_map' not in locals() or not knowledge_map: raise NameError("'knowledge_map' not loaded or empty.")

# --- Funkcija dlja sravnenija ---
def compare_weights(param_name: str, transpose_hf: bool = False):
    """
    Sravnivaet ves iz original'noj modeli HF s vesom iz Veector knowledge tensor.

    Args:
        param_name (str): Imja parametra (kak v HF modeli).
        transpose_hf (bool): Nuzhno li transponirovat' ves iz HF modeli dlja sravnenija.
                             True dlja vesov linejnyh sloev (krome embedding).
    """
    print(f"\n--- Comparing: {param_name} ---")
    knowledge_id = knowledge_map.get(param_name)
    if not knowledge_id:
        print(f"  ERROR: Knowledge ID not found in knowledge_map for '{param_name}'")
        return

    try:
        # 1. Zagruzhaem original'nyj ves HF
        param = model.get_parameter(param_name)
        hf_weight = param.data.cpu().to(torch.float32).numpy()
        if transpose_hf:
            # Transponiruem dlja sootvetstvija NumPy matmul (input @ weight)
            # Esli ishodnyj ves (out, in), delaem (in, out)
            if hf_weight.ndim == 2: # Tol'ko dlja 2D vesov
                 hf_weight = hf_weight.T
            else:
                 print(f"  WARN: Transpose requested for non-2D HF weight ({param_name}, shape {hf_weight.shape}) - skipping transpose.")

        print(f"  HF Weight: shape={hf_weight.shape}, dtype={hf_weight.dtype}, mean={np.mean(hf_weight):.4e}, std={np.std(hf_weight):.4e}")

        # 2. Zagruzhaem ves iz Veector (s dekvantovaniem, esli nuzhno)
        veector_weight = vec.load_knowledge_tensor_data(knowledge_id)

        if veector_weight is None:
            print(f"  ERROR: Failed to load Veector knowledge data for ID: {knowledge_id}")
            return

        # load_knowledge_tensor_data uzhe vozvrashhaet float32 posle dekvantovanija
        print(f"  Veector Weight: shape={veector_weight.shape}, dtype={veector_weight.dtype}, mean={np.mean(veector_weight):.4e}, std={np.std(veector_weight):.4e}")

        # 3. Sravnivaem
        if hf_weight.shape != veector_weight.shape:
            print(f"  ERROR: Shape mismatch! HF={hf_weight.shape}, Veector={veector_weight.shape}")
            # Mozhno popytat'sja sravnit' posle reshape/transpose, esli eto ozhidaemo
            # if transpose_hf and hf_weight.T.shape == veector_weight.shape:
            #     hf_weight = hf_weight.T
            #     print("  INFO: Transposed HF weight for comparison attempt.")
            # else:
            #     return # Ne sravnivaem, esli formy raznye
            return # Poka prosto vyhodim pri nesootvetstvii form

        max_abs_diff = np.max(np.abs(hf_weight - veector_weight))
        mean_abs_diff = np.mean(np.abs(hf_weight - veector_weight))

        print(f"  Comparison (float32):")
        print(f"    Max Abs Difference:  {max_abs_diff:.4e}")
        print(f"    Mean Abs Difference: {mean_abs_diff:.4e}")

        # Proverka na bol'shie rashozhdenija
        # Porogi nuzhno podbirat' (zavisjat ot float16/int8)
        if max_abs_diff > 1e-2 or mean_abs_diff > 1e-3:
             print(f"  !!! WARNING: Significant difference detected !!!")

    except Exception as e:
        print(f"  ERROR during comparison for '{param_name}': {e}")
        traceback.print_exc()

# --- Vyzyvaem sravnenie dlja neskol'kih kljuchevyh vesov ---

# Embedding (ne transponiruem, kvantovan v int8)
compare_weights("model.embed_tokens.weight", transpose_hf=False)

# Q-proekcija pervogo sloja (transponiruem, float16)
compare_weights("model.layers.0.self_attn.q_proj.weight", transpose_hf=True)

# Gate-proekcija FFN pervogo sloja (transponiruem, float16)
compare_weights("model.layers.0.mlp.gate_proj.weight", transpose_hf=True)

# Ves poslednego LayerNorm (ne transponiruem, float16)
compare_weights("model.norm.weight", transpose_hf=False)

# LM Head (transponiruem, kvantovan v int8)
compare_weights("lm_head.weight", transpose_hf=True)

# Mozhno dobavit' drugie vesovye matrichy dlja bolee detal'noj proverki
# compare_weights("model.layers.10.self_attn.v_proj.weight", transpose_hf=True)
# compare_weights("model.layers.27.mlp.down_proj.weight", transpose_hf=True)

print("\n--- Tensor Verification Finished ---")




--- Running Cell 5.6: Verify Tensor Data ---

--- Comparing: model.embed_tokens.weight ---
  HF Weight: shape=(151936, 1536), dtype=float32, mean=3.8134e-04, std=3.0062e-02
  Veector Weight: shape=(151936, 1536), dtype=float32, mean=3.8103e-04, std=3.0068e-02
  Comparison (float32):
    Max Abs Difference:  1.2995e-03
    Mean Abs Difference: 6.4934e-04

--- Comparing: model.layers.0.self_attn.q_proj.weight ---
  HF Weight: shape=(1536, 1536), dtype=float32, mean=1.7502e-05, std=5.3037e-02
  Veector Weight: shape=(1536, 1536), dtype=float16, mean=1.7524e-05, std=5.3070e-02
  Comparison (float32):
    Max Abs Difference:  0.0000e+00
    Mean Abs Difference: 0.0000e+00

--- Comparing: model.layers.0.mlp.gate_proj.weight ---
  HF Weight: shape=(1536, 8960), dtype=float32, mean=-4.8338e-07, std=3.8473e-02
  Veector Weight: shape=(1536, 8960), dtype=float16, mean=-4.7684e-07, std=3.9093e-02
  Comparison (float32):
    Max Abs Difference:  0.0000e+00
    Mean Abs Difference: 0.0000e+00

---

In [11]:
# === Cell 5.7: Verify Tensor Data (Fixed is_connected check) ===
import numpy as np
import torch
import traceback
from pathlib import Path # Dobavim import Path, esli ego net vyshe v jachejke
import pickle # Dobavim import pickle dlja zagruzki knowledge_map

print("\n--- Running Cell 5.7: Verify Tensor Data ---")

# --- Ubedimsja, chto DB_PATH opredelen (nuzhno dlja perezagruzki vec, esli nado) ---
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
    # Poprobuem ego poluchit' iz HF_MODEL_NAME, esli on est'
    if 'HF_MODEL_NAME' in locals() and HF_MODEL_NAME:
         # Predpolagaem standartnuju strukturu puti
         script_dir_simulated = Path('.') # V Colab eto /content
         DB_PATH = script_dir_simulated / "data" / "db"
         print(f"WARN: DB_PATH not found, assuming default: {DB_PATH}")
         DB_PATH.mkdir(parents=True, exist_ok=True) # Sozdaem, esli net
    else:
         raise NameError("DB_PATH not defined.")


# --- Proverka nalichija peremennyh ---
if 'model' not in locals() or model is None: raise NameError("HF 'model' not loaded. Run previous cells.")
if 'HF_MODEL_NAME' not in locals() or not HF_MODEL_NAME: raise NameError("HF_MODEL_NAME not defined.")

# --- Zagruzka knowledge_map (esli ne zagruzhen) ---
if 'knowledge_map' not in locals() or not knowledge_map:
    print("Attempting to load knowledge_map...")
    knowledge_map_filename = f"{HF_MODEL_NAME}_knowledge_map.pkl"
    knowledge_map_filepath = DB_PATH / knowledge_map_filename
    if knowledge_map_filepath.is_file():
        try:
            with open(knowledge_map_filepath, 'rb') as f:
                knowledge_map = pickle.load(f)
            print(f"Loaded knowledge_map from {knowledge_map_filepath}")
            if not knowledge_map: raise ValueError("Loaded knowledge_map is empty.")
        except Exception as e:
             raise FileNotFoundError(f"Failed to load knowledge_map from {knowledge_map_filepath}: {e}")
    else:
         raise FileNotFoundError(f"knowledge_map file not found: {knowledge_map_filepath}. Run Cell 5 first.")

# --- Inicializacija Veector (esli nuzhno) ---
if 'vec' not in locals() or vec is None:
    print("Attempting to initialize Veector...")
    try:
        # Importiruem zavisimosti dlja Veector
        from core import Veector
        from veectordb import VeectorDB # Importiruem zdes', tak kak mozhet byt' ne nuzhno vyshe
        vec = Veector(db_dir=DB_PATH)
        print("Veector initialized.")
    except Exception as e:
        raise RuntimeError(f"Failed to initialize Veector: {e}")
# --- ИСПРАВЛЕНО: Убрана проверка is_connected() ---
elif not hasattr(vec, 'db') or vec.db is None:
     print("Attempting to re-initialize Veector DB object...")
     try:
         from veectordb import VeectorDB
         # Prosto pereinicializiruem db, esli on None ili otsutstvuet
         vec.db = VeectorDB(db_dir=DB_PATH)
         print("Veector DB object re-initialized.")
     except Exception as db_reinit_e:
         raise AttributeError(f"DB re-init failed: {db_reinit_e}")
# --- КОНЕЦ ИСПРАВЛЕНИЯ ---


# --- Funkcija dlja sravnenija ---
def compare_weights(param_name: str, transpose_hf: bool = False):
    """
    Sravnivaet ves iz original'noj modeli HF s vesom iz Veector knowledge tensor.
    """
    print(f"\n--- Comparing: {param_name} ---")
    knowledge_id = knowledge_map.get(param_name)
    if not knowledge_id:
        print(f"  ERROR: Knowledge ID not found in knowledge_map for '{param_name}'")
        return

    try:
        # 1. Zagruzhaem original'nyj ves HF
        # Ubedimsja, chto model' na CPU dlja konvertacii v NumPy
        param = model.get_parameter(param_name)
        hf_weight = param.data.cpu().to(torch.float32).numpy()
        original_hf_shape = hf_weight.shape # Sohranim original'nuju formu

        if transpose_hf:
            if hf_weight.ndim == 2:
                 hf_weight = hf_weight.T
            else:
                 print(f"  WARN: Transpose requested for non-2D HF weight ({param_name}, shape {original_hf_shape}) - skipping transpose.")

        print(f"  HF Weight (as float32, {'TRANSPOSED' if transpose_hf and hf_weight.ndim==2 else 'original'}): shape={hf_weight.shape}, dtype={hf_weight.dtype}, mean={np.mean(hf_weight):.4e}, std={np.std(hf_weight):.4e}")

        # 2. Zagruzhaem ves iz Veector (s dekvantovaniem, esli nuzhno)
        # Predpolagaem, chto vec.db uzhe inicializirovan
        veector_weight = vec.load_knowledge_tensor_data(knowledge_id)

        if veector_weight is None:
            print(f"  ERROR: Failed to load Veector knowledge data for ID: {knowledge_id}")
            return

        # load_knowledge_tensor_data dolzhen vernut' float32
        print(f"  Veector Weight (loaded as float32): shape={veector_weight.shape}, dtype={veector_weight.dtype}, mean={np.mean(veector_weight):.4e}, std={np.std(veector_weight):.4e}")

        # 3. Sravnivaem
        if hf_weight.shape != veector_weight.shape:
            print(f"  ERROR: Shape mismatch! HF={hf_weight.shape}, Veector={veector_weight.shape}")
            # Dopolnitel'naja proverka: mozhet, nuzhno bylo transponirovat' HF, a my ne sdelali?
            if not transpose_hf and hf_weight.ndim == 2 and hf_weight.T.shape == veector_weight.shape:
                 print(f"  INFO: Trying comparison after transposing original HF weight...")
                 hf_weight = hf_weight.T
                 # Povtorjaem sravnenie posle transpozicii
                 max_abs_diff = np.max(np.abs(hf_weight - veector_weight))
                 mean_abs_diff = np.mean(np.abs(hf_weight - veector_weight))
                 print(f"  Comparison (float32, HF Transposed):")
                 print(f"    Max Abs Difference:  {max_abs_diff:.4e}")
                 print(f"    Mean Abs Difference: {mean_abs_diff:.4e}")
                 if max_abs_diff > 1e-2 or mean_abs_diff > 1e-3: print(f"  !!! WARNING: Significant difference detected !!!")

            return # Vyhodim, esli formy vse ravno ne sovpali

        # Esli formy sovpali srazu
        max_abs_diff = np.max(np.abs(hf_weight - veector_weight))
        mean_abs_diff = np.mean(np.abs(hf_weight - veector_weight))

        print(f"  Comparison (float32):")
        print(f"    Max Abs Difference:  {max_abs_diff:.4e}")
        print(f"    Mean Abs Difference: {mean_abs_diff:.4e}")

        # Proverka na bol'shie rashozhdenija
        # Porogi primernye, mogut zaviset' ot tochnosti (float16 vs int8)
        threshold = 1e-2 # Porog dlja maksimal'noj raznicy
        mean_threshold = 1e-3 # Porog dlja srednej raznicy
        if max_abs_diff > threshold or mean_abs_diff > mean_threshold:
             print(f"  !!! WARNING: Significant difference detected (Max > {threshold:.1e} or Mean > {mean_threshold:.1e}) !!!")

    except Exception as e:
        print(f"  ERROR during comparison for '{param_name}': {e}")
        traceback.print_exc()

# --- Vyzyvaem sravnenie dlja neskol'kih kljuchevyh vesov ---

# Embedding (ne transponiruem, kvantovan v int8)
compare_weights("model.embed_tokens.weight", transpose_hf=False)

# Q-proekcija pervogo sloja (transponiruem, float16)
compare_weights("model.layers.0.self_attn.q_proj.weight", transpose_hf=True)

# Gate-proekcija FFN pervogo sloja (transponiruem, float16)
compare_weights("model.layers.0.mlp.gate_proj.weight", transpose_hf=True)

# Ves poslednego LayerNorm (ne transponiruem, float16)
compare_weights("model.norm.weight", transpose_hf=False)

# LM Head (transponiruem, kvantovan v int8)
compare_weights("lm_head.weight", transpose_hf=True)

# --- Dopolnitel'nye proverki (mozhno raskommentirovat') ---
compare_weights("model.layers.0.self_attn.k_proj.weight", transpose_hf=True)
compare_weights("model.layers.0.self_attn.v_proj.weight", transpose_hf=True)
compare_weights("model.layers.0.self_attn.o_proj.weight", transpose_hf=True)
compare_weights("model.layers.0.mlp.up_proj.weight", transpose_hf=True)
compare_weights("model.layers.0.mlp.down_proj.weight", transpose_hf=True)
compare_weights("model.layers.0.input_layernorm.weight", transpose_hf=False)
compare_weights("model.layers.0.post_attention_layernorm.weight", transpose_hf=False)

print("\n--- Tensor Verification Finished ---")

# Vazhno: zakryt' soedinenie s BD posle proverki, esli ono bylo otkryto zdes'
# Ne zakryvaem avtomaticheski, tak kak 'vec' mozhet byt' ispol'zovan dalshe
# if 'vec' in locals() and vec and hasattr(vec, 'db') and vec.db and vec.db.is_connected():
#      print("\nClosing DB connection after verification...")
#      vec.db.close()




--- Running Cell 5.7: Verify Tensor Data ---

--- Comparing: model.embed_tokens.weight ---
  HF Weight (as float32, original): shape=(151936, 1536), dtype=float32, mean=3.8134e-04, std=3.0062e-02
  Veector Weight (loaded as float32): shape=(151936, 1536), dtype=float32, mean=3.8103e-04, std=3.0068e-02
  Comparison (float32):
    Max Abs Difference:  1.2995e-03
    Mean Abs Difference: 6.4934e-04

--- Comparing: model.layers.0.self_attn.q_proj.weight ---
  HF Weight (as float32, TRANSPOSED): shape=(1536, 1536), dtype=float32, mean=1.7502e-05, std=5.3037e-02
  Veector Weight (loaded as float32): shape=(1536, 1536), dtype=float16, mean=1.7524e-05, std=5.3070e-02
  Comparison (float32):
    Max Abs Difference:  0.0000e+00
    Mean Abs Difference: 0.0000e+00

--- Comparing: model.layers.0.mlp.gate_proj.weight ---
  HF Weight (as float32, TRANSPOSED): shape=(1536, 8960), dtype=float32, mean=-4.8338e-07, std=3.8473e-02
  Veector Weight (loaded as float32): shape=(1536, 8960), dtype=float16, 

In [13]:
!rm -rf data/db/g500

In [14]:
# === Cell 6: Define Processor Tensors (KV Placeholders) ===

import pickle
import time
import numpy as np
import traceback
import os
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple, Union

# --- Версия ---
# Obnovili versiju, chtoby otrazit' izmenenija dlja KV
CONVERTER_CELL6_VERSION = "Creating ALL Layer Processors v0.7.13 - KV Placeholders"
# --- Конец Версии ---

print(f"\n--- Running Converter Cell 6 v{CONVERTER_CELL6_VERSION} ---")
start_cell6_time = time.time()

# --- Загрузка промежуточных данных ---
print("--- Loading Intermediate Data for Cell 6 ---")
# Predpolagaem, chto HF_MODEL_NAME i DB_PATH opredeleny rannee
if 'HF_MODEL_NAME' not in locals() or not HF_MODEL_NAME:
    raise NameError("HF_MODEL_NAME not defined.")
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
    raise NameError("DB_PATH not defined or invalid.")

intermediate_filename = f"{HF_MODEL_NAME}_cell6_input_data.pkl"
intermediate_filepath = DB_PATH / intermediate_filename
cell6_input_data: Optional[Dict] = None
knowledge_map: Optional[Dict[str, str]] = None
model_config: Optional[Any] = None # Tip zavisit ot transformers

try:
    print(f"Attempting to load data from: {intermediate_filepath}")
    if not intermediate_filepath.is_file():
        raise FileNotFoundError("Intermediate data file not found. Run Cell 5.5.")
    with open(intermediate_filepath, 'rb') as f:
        cell6_input_data = pickle.load(f)

    if not isinstance(cell6_input_data, dict):
        raise TypeError("Loaded intermediate data is not a dictionary.")

    knowledge_map = cell6_input_data.get('knowledge_map')
    model_config = cell6_input_data.get('model_config')
    loaded_hf_model_name = cell6_input_data.get('hf_model_name')
    loaded_db_path_str = cell6_input_data.get('db_path')

    if not isinstance(knowledge_map, dict) or not knowledge_map:
        raise ValueError("'knowledge_map' missing or invalid in loaded data.")
    if model_config is None:
        raise ValueError("'model_config' missing in loaded data.")
    if loaded_hf_model_name != HF_MODEL_NAME:
        print(f"Warning: HF_MODEL_NAME mismatch ('{HF_MODEL_NAME}' vs loaded"
              f" '{loaded_hf_model_name}'). Using '{HF_MODEL_NAME}'.")
    if loaded_db_path_str != str(DB_PATH.resolve()):
        print(f"Warning: DB_PATH mismatch ('{str(DB_PATH.resolve())}' vs loaded"
              f" '{loaded_db_path_str}'). Using '{str(DB_PATH.resolve())}'.")

    print("Intermediate data loaded successfully.")
    print(f"  Knowledge map entries: {len(knowledge_map)}")
    print(f"  Model Config Type: {type(model_config)}")

except Exception as load_e:
    print(f"---!!! FATAL ERROR loading intermediate data: {load_e} !!!---")
    raise

# --- Poluchaem parametry iz konfiga ---
required_attrs = [
    'num_hidden_layers', 'num_attention_heads',
    'num_key_value_heads', 'hidden_size'
]
if not all(hasattr(model_config, attr) for attr in required_attrs):
    raise AttributeError("Loaded model_config missing required attributes.")

num_layers: int = model_config.num_hidden_layers
num_attention_heads: int = model_config.num_attention_heads
num_key_value_heads: int = model_config.num_key_value_heads
hidden_size: int = model_config.hidden_size
if num_attention_heads == 0:
    raise ValueError("num_attention_heads cannot be zero.")
head_dim: int = hidden_size // num_attention_heads
if hidden_size % num_attention_heads != 0:
     print(f"WARN: hidden_size {hidden_size} not perfectly divisible by num_attention_heads {num_attention_heads}")

print(f"Using config: L={num_layers}, H={num_attention_heads}, "
      f"H_KV={num_key_value_heads}, HDim={head_dim}")
# --- Konec Zagruzki ---


# --- Importy i Proverki Versij ---
# Ozhidaemye versii: core >= 0.7.2, tensors >= 0.7.6, operations >= 0.8.3
TENSORS_VERSION_REQ_CELL6 = "0.7.6"
CORE_VERSION_REQ_CELL6 = "0.7.2" # Versija s obrabotkoj kortezha
OPERATIONS_VERSION_REQ_CELL6 = "0.8.3" # Versija s novymi operacijami
try:
    from tensors import (
        TENSORS_VERSION, TensorCoordinate, create_tensor, MetadataTuple,
        validate_tensor, validate_tensor_tuple, get_tensor_hash,
        get_tensor_metadata, get_tensor_coord, get_tensor_tags,
        TAG_TYPE_PROCESSOR, TAG_FUNC_EMBED_LOOKUP, TAG_FUNC_ATTENTION,
        TAG_FUNC_FFN, TAG_FUNC_LINEAR, TAG_COMP_LAYERNORM, TAG_MODEL_DEEPSEEK,
        tag_layer, GROUP_IDX_QWEN_PROCESSOR, TAG_COMP_EMBEDDING,
        TAG_COMP_WEIGHTS, TAG_COMP_BIAS, TAG_COMP_ATTN_Q, TAG_COMP_ATTN_K,
        TAG_COMP_ATTN_V, TAG_COMP_ATTN_O, TAG_COMP_FFN_GATE, TAG_COMP_FFN_UP,
        TAG_COMP_FFN_DOWN, TAG_COMP_LM_HEAD, TAG_PREC_INT8, TAG_PREC_FLOAT16
    )
    if TENSORS_VERSION < TENSORS_VERSION_REQ_CELL6:
        raise ImportError(f"Requires tensors v{TENSORS_VERSION_REQ_CELL6}+")
    from core import Veector, CORE_VERSION
    if CORE_VERSION < CORE_VERSION_REQ_CELL6:
        raise ImportError(f"Requires core v{CORE_VERSION_REQ_CELL6}+")
    from operations import OPERATIONS_VERSION
    if OPERATIONS_VERSION < OPERATIONS_VERSION_REQ_CELL6:
        raise ImportError(f"Requires operations v{OPERATIONS_VERSION_REQ_CELL6}+")

    # --- Lokal'noe opredelenie OP kodov (VKLUCHAJA NOVYE) ---
    OP_SUM=[0,0,0]; OP_SUBTRACT=[0,0,1]; OP_ADD=[0,0,2]; OP_MULTIPLY=[0,1,0]
    OP_DIVIDE=[0,1,1]; OP_SQRT=[0,2,0]; OP_POWER=[0,2,1]; OP_ABS=[0,3,0]
    OP_MOD=[0,5,0]; OP_FLOOR=[0,6,0]; OP_CEIL=[0,6,1]; OP_SIN=[1,0,0]
    OP_COS=[1,0,1]; OP_TAN=[1,1,0]; OP_COT=[1,1,1]; OP_ASIN=[1,2,0]
    OP_ACOS=[1,2,1]; OP_ATAN=[1,3,0]; OP_GREATER=[2,0,0]; OP_EQUAL=[2,0,1]
    OP_AND=[2,1,0]; OP_OR=[2,1,1]; OP_NOT=[2,2,0]; OP_XOR=[2,3,0]
    OP_NAND=[2,4,0]; OP_NOR=[2,4,1]; OP_IF=[3,0,0]; OP_LOOP_MULT=[4,0,0]
    OP_CHOICE=[7,0,0]; OP_RAND_UNIFORM=[5,1,0]; OP_RAND_NORMAL=[5,1,1]
    OP_MEDIAN=[5,2,0]; OP_PRINT=[8,0,0]; OP_IDENTITY=[9,0,0]
    OP_TRIGGER_REASON=[10,0,0]; OP_DFS=[15,0,0]; OP_MEAN=[16,0,0]
    OP_STDDEV=[16,1,0]; OP_RELU=[18,0,0]; OP_SIGMOID=[18,1,0]
    OP_SOFTMAX=[18,2,0]; OP_LEAKY_RELU=[18,3,0]; OP_SILU=[18,4,0]
    OP_GELU=[40,5,0]; OP_EXP_SMOOTHING=[19,0,0]; OP_NORMALIZE_01=[20,0,0]
    OP_INTERPOLATE=[20,1,0]; OP_LAYER_NORM=[40,1,0]; OP_BATCH_NORM=[40,4,0]
    OP_DROPOUT=[40,3,0]; OP_MATRIX_MULTIPLY=[30,0,0]; OP_DETERMINANT=[30,1,0]
    OP_EIGENVALUES=[30,2,0]; OP_CONVOLUTION=[30,3,0]; OP_TRANSPOSE=[30,4,0]
    OP_INVERSE=[30,5,0]; OP_TRACE=[30,6,0]; OP_ATTENTION_MULTIHEAD=[40,2,0]
    OP_EMBEDDING_LOOKUP=[40,6,0]; OP_APPLY_ROPE=[40,7,0]; OP_GET_Q_ROT=[40,7,1]
    OP_GET_K_ROT=[40,7,2]; OP_RESHAPE_HEADS=[40,9,0]; OP_REPEAT_KV_HEADS=[40,9,1]
    OP_SCALED_DOT_PROD_ATTN=[40,9,2]; OP_MERGE_HEADS=[40,9,3]; OP_ADD_BIAS=[0,0,3]
    OP_UPDATE_KV_CACHE = [40, 10, 0]    # <<< Dobavleno
    OP_CREATE_CAUSAL_MASK = [40, 10, 1] # <<< Dobavleno
    OP_RESIDUAL_ADD=OP_ADD; OP_LINEAR=OP_MATRIX_MULTIPLY; OP_FINAL_NORM=OP_LAYER_NORM
    OP_LINEAR_HEAD=OP_LINEAR; OP_QUANTUM_HADAMARD=[50,0,0]; OP_QUANTUM_PAULI_X=[50,0,1]
    OP_QUANTUM_CNOT=[50,1,0]; OP_QUANTUM_MEASURE=[50,2,0]; OP_QUANTUM_SUPERPOS=[50,3,0]
    OP_QUANTUM_ENTANGLE=[50,4,0]; META_OP_CATEGORY=99; OP_STORE=[99,0,0]
    OP_LOAD=[99,0,1]; OP_LOAD_INITIAL_INPUT=[99,0,3]; OP_DEBUG_CONTEXT=[99,1,0]
    OP_MAKE_TUPLE = [99, 2, 0]         # <<< Dobavleno
    # --- Konec OP kodov ---

except ImportError as e: print(f"FATAL ERROR imports: {e}"); raise
except Exception as e_other: print(f"FATAL ERROR imports: {e_other}"); traceback.print_exc(); raise


# --- Ochistka staroj karty processorov ---
processor_map_filename = f"{HF_MODEL_NAME}_proc_map.pkl"
processor_map_filepath = DB_PATH / processor_map_filename
if processor_map_filepath.is_file():
    try:
        print(f"Deleting existing processor map: {processor_map_filepath}")
        os.remove(processor_map_filepath)
    except OSError as e:
        print(f"Warning: Could not delete existing processor map: {e}")

# --- Inicializacija Veector i dr. ---
print("--- Defining ALL Processor Tensors (Embedding, Layers 0-27, FinalNorm, LMHead) ---")
if 'vec' not in locals() or vec is None or not hasattr(vec, 'db') or vec.db is None:
     print("Re-initializing Veector for Cell 6...")
     try:
         vec = Veector(db_dir=DB_PATH)
         print(f"Veector core v{CORE_VERSION} initialized for Cell 6")
     except Exception as e: print(f"FATAL: Veector re-init failed: {e}"); raise
else:
     print("Using existing Veector object.")

processor_errors = 0
processor_map: Dict[str, str] = {}
knowledge_map_by_name = knowledge_map # Ispol'zuem zagruzhennuju kartu
def find_knowledge_id(pattern: str) -> Optional[str]:
    return knowledge_map_by_name.get(pattern)

processor_group_idx = GROUP_IDX_QWEN_PROCESSOR # 500
model_tag = TAG_MODEL_DEEPSEEK # 12
prec_tag_weights = TAG_PREC_FLOAT16 # 21

# --- 1. Embedding Processor ---
proc_name = "Embedding Processor"
print(f"\n--- Defining {proc_name} ---")
try:
    coord = TensorCoordinate(layer=-1, group=processor_group_idx, nest=0, x=0)
    tags = [TAG_TYPE_PROCESSOR, TAG_FUNC_EMBED_LOOKUP, model_tag]
    param_name = "embedding_matrix"; kn_tags = [TAG_COMP_EMBEDDING, model_tag, TAG_COMP_WEIGHTS, TAG_PREC_INT8] # Prec INT8
    interface = {"inputs": [{"name":"token_ids","dtype":"int64"}], "outputs": [{"name":"hidden_states","dtype":"float16"}], "knowledge_needed": [{"param_name": param_name, "tags": kn_tags}]}
    ops_sequences = {'default': [ [OP_EMBEDDING_LOOKUP, {"embedding_matrix": param_name}] ]}
    tensor_structure = vec.create_tensor(coord=coord, tensor_type="processor", tags=tags, interface=interface, ops_sequences=ops_sequences, status="active", name_id=-1)
    if not validate_tensor(tensor_structure): raise ValueError("Invalid list structure (Embed)")
    proc_id = vec.save_tensor(tensor_structure);
    if proc_id: processor_map["embedding"] = proc_id; print(f"  SUCCESS: Saved {proc_name}: {proc_id}")
    else: processor_errors += 1; print(f"  ERROR saving {proc_name}")
except Exception as e: print(f"  ERROR during {proc_name}: {e}"); traceback.print_exc(); processor_errors += 1


# --- 2. Cikl po Slojam Transformera ---
print(f"\n--- Defining Transformer Layer Processors (0 to {num_layers-1}) ---")
for layer_idx in range(num_layers):
    # --- Nachalo bloka dlja Attention Processor L{layer_idx} ---
    proc_attn_name = f"Attention Processor L{layer_idx}"
    print(f"\n--- Defining {proc_attn_name} ---")
    layer_tag = tag_layer(layer_idx)

    # --- Opredeljaem neobhodimye znanija dlja etogo processora ---
    kn_defs_attn = [
        {"p":"norm_weight", "t":[TAG_COMP_LAYERNORM, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f":f"model.layers.{layer_idx}.input_layernorm.weight"},
        {"p":"q_weights",   "t":[TAG_COMP_ATTN_Q, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f":f"model.layers.{layer_idx}.self_attn.q_proj.weight"},
        {"p":"q_bias",      "t":[TAG_COMP_ATTN_Q, layer_tag, model_tag, TAG_COMP_BIAS, prec_tag_weights],    "f":f"model.layers.{layer_idx}.self_attn.q_proj.bias", "opt": True},
        {"p":"k_weights",   "t":[TAG_COMP_ATTN_K, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f":f"model.layers.{layer_idx}.self_attn.k_proj.weight"},
        {"p":"k_bias",      "t":[TAG_COMP_ATTN_K, layer_tag, model_tag, TAG_COMP_BIAS, prec_tag_weights],    "f":f"model.layers.{layer_idx}.self_attn.k_proj.bias", "opt": True},
        {"p":"v_weights",   "t":[TAG_COMP_ATTN_V, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f":f"model.layers.{layer_idx}.self_attn.v_proj.weight"},
        {"p":"v_bias",      "t":[TAG_COMP_ATTN_V, layer_tag, model_tag, TAG_COMP_BIAS, prec_tag_weights],    "f":f"model.layers.{layer_idx}.self_attn.v_proj.bias", "opt": True},
        {"p":"o_weights",   "t":[TAG_COMP_ATTN_O, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f":f"model.layers.{layer_idx}.self_attn.o_proj.weight"},
    ]
    knowledge_needs_attn = [] # Inicializiruem pustoj spisok
    missing_essential_attn = False
    for kdef in kn_defs_attn:
        kid = find_knowledge_id(kdef["f"])
        is_opt = kdef.get("opt", False)
        if kid:
            knowledge_needs_attn.append({"param_name": kdef["p"], "tags": kdef["t"], "optional": is_opt})
        elif not is_opt:
            missing_essential_attn = True
            processor_errors += 1
            print(f"      ERROR: Missing Attn L{layer_idx} knowledge: {kdef['p']} (pattern: {kdef['f']})")

    # --- Opredeljaem interface s vhodami/vyhodami dlja KV kjesha ---
    interface_attn = {
        "inputs": [
            {"name": "hidden_state_in"},
            {"name": "residual_input"},
            {"name": "position_ids"},
            {"name": "attention_mask", "optional": True}, # Budet kauzal'naja maska
            {"name": "past_key", "optional": True},
            {"name": "past_value", "optional": True},
            {"name": "start_pos", "dtype": "int", "optional": True},
            {"name": "total_seq_len", "dtype": "int", "optional": True}, # Nuzhno dlja kauzal'noj maski
        ],
        "outputs": [
            # Vyhod teper' kortezh: (attn_output, new_key, new_value)
            {"name": "attn_processor_output_tuple"}
        ],
        "knowledge_needed": knowledge_needs_attn
    }

    # --- Opredeljaem posledovatel'nost' operacij s KV keshom ---
    ops_sequences_attn = {'default': [
        # --- Nachalo bloka Attention ---
        [OP_STORE, 'attn_residual_input'],                      # 0: Sohranjaem vhod dlja residual connection
        [OP_LAYER_NORM, {"norm_weight": "norm_weight"}],       # 1: Normalizacija vhoda
        [OP_STORE, 'norm_output'],                              # 2: Sohranjaem normalizovannyj vyhod

        # --- Vychislenie Q (dlja tekushhego shaga) ---
        [OP_LOAD, 'norm_output'],                               # 3
        [OP_LINEAR, {"weights": "q_weights", "bias": "q_bias"}],# 4
        [OP_RESHAPE_HEADS, {"num_heads": num_attention_heads}], # 5
        [OP_STORE, 'q_proj_4d'],                                # 6

        # --- Vychislenie K (dlja tekushhego shaga) ---
        [OP_LOAD, 'norm_output'],                               # 7
        [OP_LINEAR, {"weights": "k_weights", "bias": "k_bias"}],# 8
        [OP_RESHAPE_HEADS, {"num_heads": num_key_value_heads}], # 9
        [OP_STORE, 'k_proj_4d'],                                # 10

        # --- Vychislenie V (dlja tekushhego shaga) ---
        [OP_LOAD, 'norm_output'],                               # 11
        [OP_LINEAR, {"weights": "v_weights", "bias": "v_bias"}],# 12
        [OP_RESHAPE_HEADS, {"num_heads": num_key_value_heads}], # 13
        [OP_STORE, 'v_proj_4d'],                                # 14

        # --- Obnovlenie V Kjesha ---
        [OP_LOAD, 'past_value'], # Zagruzhaem staryj kesh V iz konteksta
        [OP_UPDATE_KV_CACHE, {"cache": "past_value", "new_values": "v_proj_4d", "start_pos": "start_pos"}], # 15 Obnovljaem kesh V tekushhimi znachenijami
        [OP_STORE, 'v_cache_out'],                              # 16 Sohranjaem obnovlennyj kesh V

        # --- RoPE (Primenjaetsja k Q i K tekushhego shaga) ---
        [OP_LOAD, 'q_proj_4d'], [OP_STORE, 'q_for_rope'],       # 17, 18
        [OP_LOAD, 'k_proj_4d'], [OP_STORE, 'k_for_rope'],       # 19, 20
        [OP_APPLY_ROPE, {"q": "q_for_rope", "k": "k_for_rope", "position_ids": "position_ids", "head_dim": head_dim }], # 21
        [OP_STORE, 'rope_output'],                              # 22
        [OP_LOAD, 'rope_output'], [OP_GET_Q_ROT], [OP_STORE, 'q_rot'],# 23,24,25 (q_rot dlja tekushhego shaga)
        [OP_LOAD, 'rope_output'], [OP_GET_K_ROT], [OP_STORE, 'k_rot'],# 26,27,28 (k_rot dlja tekushhego shaga)

        # --- Obnovlenie K Kjesha (POSLE RoPE) ---
        [OP_LOAD, 'past_key'],   # Zagruzhaem staryj kesh K iz konteksta
        [OP_UPDATE_KV_CACHE, {"cache": "past_key", "new_values": "k_rot", "start_pos": "start_pos"}], # 29 Obnovljaem kesh K tekushhimi znachenijami (posle RoPE)
        [OP_STORE, 'k_cache_out'],                              # 30 Sohranjaem obnovlennyj kesh K

        # --- Podgotovka k SDPA ---
        # Sozdaem kauzal'nuju masku dlja tekushhej polnoj dliny
        # Predpolagaem, chto 'total_seq_len' peredan v kontexte
        [OP_CREATE_CAUSAL_MASK, {"size": "total_seq_len"}],     # 31
        [OP_STORE, 'causal_mask'],                              # 32

        # --- Attention (SDPA) - Teper' ispol'zuem KESH i kauzal'nuju masku ---
        # VAZHNO: SDPA dolzhen umet' rabotat' s polnym keshem K/V i kauzal'noj maskoj.
        # Predpolagaem, chto on beret q_rot (tekushhij) i k_cache_out/v_cache_out (polnyj kesh)
        # i pravil'no ispol'zuet kauzal'nuju masku.
        # Eto mozhet potrebovat' modifikacii samoj funkcii scaled_dot_product_attention
        # ili dobavlenija shagov OP_SLICE_CACHE pered ee vyzovom.
        [OP_SCALED_DOT_PROD_ATTN, {
            "query": "q_rot",           # Q tekushhego shaga (posle RoPE)
            "key": "k_cache_out",       # Polnyj obnovlennyj K kesh
            "value": "v_cache_out",     # Polnyj obnovlennyj V kesh
            "attention_mask": "causal_mask" # Kauzal'naja maska
        }],                                                     # 33
        [OP_STORE, 'attn_context'],                             # 34

        # --- Output Projection ---
        [OP_LOAD, 'attn_context'],                              # 35
        [OP_LINEAR, {"weights": "o_weights"}],                  # 36
        [OP_STORE, 'attn_output_proj'],                         # 37

        # --- Residual Add ---
        [OP_LOAD, 'attn_residual_input'], [OP_STORE, 'input_a'],# 38, 39
        [OP_LOAD, 'attn_output_proj'], [OP_STORE, 'input_b'],   # 40, 41
        [OP_ADD, {"input_a": "input_a", "input_b": "input_b"}], # 42
        [OP_STORE, 'final_attn_output'],                        # 43

        # --- Vozvrat rezul'tata i kjesha ---
        [OP_MAKE_TUPLE, { # Formiruem kortezh dlja vozvrata
            "elem0": "final_attn_output", # Rezul'tat sloja
            "elem1": "k_cache_out",       # Obnovlennyj K kesh
            "elem2": "v_cache_out"        # Obnovlennyj V kesh
        }]                                                      # 44
    ]}

    # --- Sozdanie i sohranenie processora Attention ---
    try:
        if not missing_essential_attn:
            coord_attn = TensorCoordinate(layer=layer_idx, group=processor_group_idx, nest=0, x=0)
            tags_attn = [TAG_TYPE_PROCESSOR, TAG_FUNC_ATTENTION, layer_tag, model_tag]
            tensor_structure_attn = vec.create_tensor(
                coord=coord_attn, tensor_type="processor", tags=tags_attn,
                interface=interface_attn, # Obnovlennyj interface
                ops_sequences=ops_sequences_attn, # Obnovlennaja posledovatel'nost'
                status="active", name_id=-1
            )
            if not validate_tensor(tensor_structure_attn): raise ValueError("Invalid list structure (Attn)")
            proc_id_attn = vec.save_tensor(tensor_structure_attn);
            if proc_id_attn:
                processor_map[f"attn_{layer_idx}"] = proc_id_attn
                print(f"  SUCCESS: Saved {proc_attn_name}: {proc_id_attn}")
            else:
                processor_errors += 1
                print(f"  ERROR saving {proc_attn_name}")
        else:
            print(f"  Skipping {proc_attn_name} due to missing essential knowledge.")
    except Exception as e:
        print(f"  ERROR during {proc_attn_name} definition: {e}")
        traceback.print_exc()
        processor_errors += 1
    # --- Konec bloka dlja Attention Processor L{layer_idx} ---


    # --- FFN Processor ---
    # (Ostavljaem FFN processor bez izmenenij na etom etape)
    proc_ffn_name = f"FFN Processor L{layer_idx}"
    print(f"\n--- Defining {proc_ffn_name} ---")
    ops_sequences_ffn = {'default': [ [OP_STORE, 'ffn_residual_input'], [OP_LAYER_NORM, {"norm_weight": "norm_weight"}], [OP_STORE, 'norm_out'], [OP_LOAD, 'norm_out'], [OP_LINEAR, {"weights": "gate_weights"}], [OP_STORE, 'gate_proj'], [OP_LOAD, 'norm_out'], [OP_LINEAR, {"weights": "up_weights"}], [OP_STORE, 'up_proj'], [OP_LOAD, 'gate_proj'], [OP_SILU], [OP_STORE, 'silu_gate'], [OP_LOAD, 'silu_gate'], [OP_STORE, 'factor1'], [OP_LOAD, 'up_proj'], [OP_STORE, 'factor2'], [OP_MULTIPLY, {"factor1": "factor1", "factor2": "factor2"}], [OP_STORE, 'activated'], [OP_LOAD, 'activated'], [OP_LINEAR, {"weights": "down_weights"}], [OP_STORE, 'ffn_out'], [OP_LOAD, 'ffn_residual_input'], [OP_STORE, 'input_a'], [OP_LOAD, 'ffn_out'], [OP_STORE, 'input_b'], [OP_ADD, {"input_a": "input_a", "input_b": "input_b"}] ]}
    try:
        coord_ffn = TensorCoordinate(layer=layer_idx, group=processor_group_idx, nest=0, x=1)
        tags_ffn = [TAG_TYPE_PROCESSOR, TAG_FUNC_FFN, layer_tag, model_tag]
        kn_defs_ffn = [
            {"p": "norm_weight",  "t": [TAG_COMP_LAYERNORM, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights], "f": f"model.layers.{layer_idx}.post_attention_layernorm.weight"},
            {"p": "gate_weights", "t": [TAG_COMP_FFN_GATE, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights],  "f": f"model.layers.{layer_idx}.mlp.gate_proj.weight"},
            {"p": "up_weights",   "t": [TAG_COMP_FFN_UP, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights],    "f": f"model.layers.{layer_idx}.mlp.up_proj.weight"},
            {"p": "down_weights", "t": [TAG_COMP_FFN_DOWN, layer_tag, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights],  "f": f"model.layers.{layer_idx}.mlp.down_proj.weight"},
        ]
        knowledge_needs_ffn = []; missing_essential_ffn = False
        for kdef in kn_defs_ffn:
            kid = find_knowledge_id(kdef["f"]); is_opt = kdef.get("opt", False)
            if kid: knowledge_needs_ffn.append({"param_name": kdef["p"], "tags": kdef["t"], "optional": is_opt})
            elif not is_opt: missing_essential_ffn = True; processor_errors += 1; print(f"      ERROR: Missing FFN L{layer_idx} knowledge: {kdef['p']}")
        if not missing_essential_ffn:
            interface_ffn = { "inputs": [{"name":"attn_output"}, {"name":"residual_input"}], "outputs": [{"name":"ffn_output"}], "knowledge_needed": knowledge_needs_ffn }
            tensor_structure_ffn = vec.create_tensor( coord=coord_ffn, tensor_type="processor", tags=tags_ffn, interface=interface_ffn, ops_sequences=ops_sequences_ffn, status="active", name_id=-1)
            if not validate_tensor(tensor_structure_ffn): raise ValueError("Invalid list structure (FFN)")
            proc_id_ffn = vec.save_tensor(tensor_structure_ffn)
            if proc_id_ffn: processor_map[f"ffn_{layer_idx}"] = proc_id_ffn; print(f"  SUCCESS: Saved {proc_ffn_name}: {proc_id_ffn}")
            else: processor_errors += 1; print(f"  ERROR saving {proc_ffn_name}")
        else: print(f"  Skipping {proc_ffn_name}")
    except Exception as e: print(f"  ERROR during {proc_ffn_name} definition: {e}"); traceback.print_exc(); processor_errors += 1
# --- KONEC CIKLA PO SLOJAM ---


# --- 3. Final Norm Processor ---
# ... (bez izmenenij) ...
proc_name = "Final Norm Processor"; print(f"\n--- Defining {proc_name} ---")
try:
    coord = TensorCoordinate(layer=-1, group=processor_group_idx, nest=0, x=1); tags = [TAG_TYPE_PROCESSOR, TAG_COMP_LAYERNORM, model_tag]; prec_tag_weights = TAG_PREC_FLOAT16
    kn_tags = [TAG_COMP_LAYERNORM, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights]; knowledge_needs = []; pattern = "model.norm.weight"; kid = find_knowledge_id(pattern)
    print(f"  Checking knowledge: Param: norm_weight, Pattern: '{pattern}', Found: {'Yes' if kid else 'No'}")
    if kid: knowledge_needs.append({"param_name": "norm_weight", "tags": kn_tags, "optional": False})
    else: processor_errors += 1; print(f"  ERROR: Essential knowledge missing!")
    if knowledge_needs:
        interface = { "inputs": [{"name":"final_hidden_state"}], "outputs": [{"name":"final_normed_state"}], "knowledge_needed": knowledge_needs }
        ops_sequences = {'default': [[OP_FINAL_NORM, {"norm_weight": "norm_weight"}]]}
        tensor_structure = vec.create_tensor( coord=coord, tensor_type="processor", tags=tags, interface=interface, ops_sequences=ops_sequences, status="active", name_id=-1)
        if not validate_tensor(tensor_structure): raise ValueError("Invalid list structure (Final Norm)")
        proc_id = vec.save_tensor(tensor_structure);
        if proc_id: processor_map["final_norm"] = proc_id; print(f"  SUCCESS: Saved {proc_name}: {proc_id}")
        else: processor_errors += 1; print(f"  ERROR saving {proc_name}")
    else: print(f"  Skipping {proc_name}")
except Exception as e: print(f"  ERROR during {proc_name} definition: {e}"); processor_errors += 1


# --- 4. LM Head Processor ---
# ... (bez izmenenij) ...
proc_name = "LM Head Processor"; print(f"\n--- Defining {proc_name} ---")
try:
    coord = TensorCoordinate(layer=-1, group=processor_group_idx, nest=0, x=2); tags = [TAG_TYPE_PROCESSOR, TAG_FUNC_LINEAR, model_tag]; prec_tag_weights = TAG_PREC_INT8
    kn_tags = [TAG_COMP_LM_HEAD, model_tag, TAG_COMP_WEIGHTS, prec_tag_weights]; knowledge_needs = []; pattern = "lm_head.weight"; kid = find_knowledge_id(pattern)
    print(f"  Checking knowledge: Param: lm_head_weights, Pattern: '{pattern}', Found: {'Yes' if kid else 'No'}")
    if kid: knowledge_needs.append({"param_name": "lm_head_weights", "tags": kn_tags, "optional": False})
    else: processor_errors += 1; print(f"  ERROR: Essential knowledge missing!")
    if knowledge_needs:
        interface = { "inputs": [{"name":"final_normed_state"}], "outputs": [{"name":"logits"}], "knowledge_needed": knowledge_needs }
        ops_sequences = {'default': [[OP_LINEAR_HEAD, {"weights": "lm_head_weights"}]]}
        tensor_structure = vec.create_tensor( coord=coord, tensor_type="processor", tags=tags, interface=interface, ops_sequences=ops_sequences, status="active", name_id=-1)
        if not validate_tensor(tensor_structure): raise ValueError("Invalid list structure (LM Head)")
        proc_id = vec.save_tensor(tensor_structure)
        if proc_id: processor_map["lm_head"] = proc_id; print(f"  SUCCESS: Saved {proc_name}: {proc_id}")
        else: processor_errors += 1; print(f"  ERROR saving {proc_name}")
    else: print(f"  Skipping {proc_name}")
except Exception as e: print(f"  ERROR during {proc_name} definition: {e}"); processor_errors += 1


# --- Zavershenie ---
# ... (bez izmenenij) ...
print(f"\n--- Processor Definition Phase Completed ({processor_errors} errors) ---")
processor_map_filename = f"{HF_MODEL_NAME}_proc_map.pkl"; processor_map_filepath = DB_PATH / processor_map_filename
try:
    if processor_errors == 0:
        expected_proc_count = 3 + 2 * num_layers
        if len(processor_map) == expected_proc_count:
             with open(processor_map_filepath, 'wb') as f: pickle.dump(processor_map, f)
             print(f"\nProcessor map saved to {processor_map_filepath} ({len(processor_map)} entries)")
        else: print(f"\nWARN: Processor map has incorrect entry count ({len(processor_map)} vs {expected_proc_count}). NOT SAVED.")
    else: print(f"\nProcessor map NOT saved due to {processor_errors} errors.")
except Exception as e: print(f"Error saving processor map: {e}")
if 'vec' in locals() and vec and hasattr(vec, 'db') and vec.db:
    print("\nExplicitly saving index and closing DB at end of Cell 6...")
    if hasattr(vec.db, '_save_index') and callable(vec.db._save_index): vec.db._save_index()
    if hasattr(vec.db, 'close') and callable(vec.db.close): vec.db.close()
    print("Index saved and DB connection closed for Cell 6.")
else: print("Warning: Could not explicitly save/close DB index at end of Cell 6.")
end_cell6_time = time.time()
print(f"--- Cell 6 Finished in {end_cell6_time - start_cell6_time:.3f} seconds ---")




--- Running Converter Cell 6 vCreating ALL Layer Processors v0.7.13 - KV Placeholders ---
--- Loading Intermediate Data for Cell 6 ---
Attempting to load data from: data/db/DeepSeek-R1-Distill-Qwen-1.5B_cell6_input_data.pkl
Intermediate data loaded successfully.
  Knowledge map entries: 339
  Model Config Type: <class 'transformers.models.qwen2.configuration_qwen2.Qwen2Config'>
Using config: L=28, H=12, H_KV=2, HDim=128
--- Defining ALL Processor Tensors (Embedding, Layers 0-27, FinalNorm, LMHead) ---
Using existing Veector object.

--- Defining Embedding Processor ---
DEBUG INDEX UPDATE: Adding/Updating ID fb769b8034ba0aae87a82183e23c06bab7160aaea5b151b94080a49f5fa7abc9 -> Type: processor, Status: active, Coords: L-1_G500_N0_X0_Y0_Z0
  SUCCESS: Saved Embedding Processor: fb769b8034ba0aae87a82183e23c06bab7160aaea5b151b94080a49f5fa7abc9

--- Defining Transformer Layer Processors (0 to 27) ---

--- Defining Attention Processor L0 ---
DEBUG INDEX UPDATE: Adding/Updating ID 637fa4ff094566

In [15]:
# === Cell 7: Analyze Veector DB Structure (Index-First Analysis v0.7.5+) ===

import os
import pickle
from pathlib import Path
import numpy as np
import time
import traceback

print(f"\n--- Running DB Analysis Cell (Index-First Analysis) ---")
start_cell7_time = time.time()

# --- Импорты из tensors.py (Нужны новые геттеры и валидатор кортежа) ---
ANALYSIS_IMPORTS_OK = False
TENSORS_VERSION_REQ = "0.7.5" # Ожидаем последнюю версию
try:
    from tensors import TENSORS_VERSION
    print(f"DEBUG: Importing from tensors version: {TENSORS_VERSION}")
    if TENSORS_VERSION < TENSORS_VERSION_REQ:
        print(f"WARN: Analysis script requires tensors v{TENSORS_VERSION_REQ}+. Found v{TENSORS_VERSION}")

    from tensors import (
        TensorCoordinate, MetadataTuple, validate_tensor_tuple, # Валидатор кортежа
        # Новые геттеры для кортежа
        get_type_code_from_meta, get_dtype_code_from_meta, get_has_blob_flag_from_meta,
        get_coord_list_from_meta, get_coord_obj_from_meta, get_tags_list_from_meta,
        # Маппинги и константы
        REVERSE_DATA_TYPE_MAPPING, DTYPE_MAPPING,
        TAG_COMP_EMBEDDING, TAG_COMP_LM_HEAD, TAG_PREC_INT8
    )
    ANALYSIS_IMPORTS_OK = True
    print("DEBUG: Imports for analysis successful.")

except ImportError as e:
    print(f"WARN: Cannot import from tensors.py for full analysis: {e}")
    ANALYSIS_IMPORTS_OK = False

    # Заглушки
    TAG_COMP_EMBEDDING = -1000
    TAG_COMP_LM_HEAD = -1001
    TAG_PREC_INT8 = -1002

    class TensorCoordinate:
        """Заглушка для класса TensorCoordinate."""
        @staticmethod
        def from_string(s):
            """Метод-заглушка. Возвращает None."""
            return None

    def validate_tensor_tuple(t):
        """Заглушка для функции validate_tensor_tuple."""
        return False

    def get_type_code_from_meta(t):
        """Заглушка для функции get_type_code_from_meta."""
        return 0

    def get_has_blob_flag_from_meta(t):
        """Заглушка для функции get_has_blob_flag_from_meta."""
        return 0

    def get_tags_list_from_meta(t):
        """Заглушка для функции get_tags_list_from_meta."""
        return []

    def get_coord_list_from_meta(t):
        """Заглушка для функции get_coord_list_from_meta."""
        return [-1] * 6

    def get_coord_obj_from_meta(t):
        """Заглушка для функции get_coord_obj_from_meta."""
        return None

    def get_dtype_code_from_meta(t):
        """Заглушка для функции get_dtype_code_from_meta."""
        return 0

    REVERSE_DATA_TYPE_MAPPING = {}
    DTYPE_MAPPING = {}

except Exception as e_other:
    print(f"WARN: Unexpected error during imports for analysis: {e_other}")
    traceback.print_exc()
    ANALYSIS_IMPORTS_OK = False

    # Заглушки
    TAG_COMP_EMBEDDING = -1000
    TAG_COMP_LM_HEAD = -1001
    TAG_PREC_INT8 = -1002

    class TensorCoordinate:
        """Заглушка для класса TensorCoordinate."""
        @staticmethod
        def from_string(s):
            """Метод-заглушка. Возвращает None."""
            return None

    def validate_tensor_tuple(t):
        """Заглушка для функции validate_tensor_tuple."""
        return False

    def get_type_code_from_meta(t):
        """Заглушка для функции get_type_code_from_meta."""
        return 0

    def get_has_blob_flag_from_meta(t):
        """Заглушка для функции get_has_blob_flag_from_meta."""
        return 0

    def get_tags_list_from_meta(t):
        """Заглушка для функции get_tags_list_from_meta."""
        return []

    def get_coord_list_from_meta(t):
        """Заглушка для функции get_coord_list_from_meta."""
        return [-1] * 6

    def get_coord_obj_from_meta(t):
        """Заглушка для функции get_coord_obj_from_meta."""
        return None

    def get_dtype_code_from_meta(t):
        """Заглушка для функции get_dtype_code_from_meta."""
        return 0

    REVERSE_DATA_TYPE_MAPPING = {}
    DTYPE_MAPPING = {}

print(f"\n--- Analyzing Veector DB Structure (Index-First Method) ---")

# --- Конфигурация ---
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
     print("ERROR: DB_PATH not defined. Assuming './data/db/'")
     DB_PATH = Path("./data/db/")

INDEX_FILENAME = "tensor_index.pkl"

if not DB_PATH.is_dir():
    print(f"ERROR: Database directory not found at {DB_PATH}")
else:
    print(f"Analyzing DB at: {DB_PATH.resolve()}")

    # Инициализация счетчиков
    total_meta_files_in_index = 0; total_meta_size = 0; total_blob_files_npy = 0
    total_blob_files_pickle = 0; total_blob_size = 0; processor_count = 0
    knowledge_count = 0; state_count = 0; converter_count = 0; other_count = 0
    error_count = 0; missing_meta_files = 0; invalid_tuples = 0
    index_entry_count = 0; index_total_size = 0; index_data = {}

    # --- 1. Загрузка Индекса ---
    index_path = DB_PATH / INDEX_FILENAME
    if not index_path.is_file(): print(f"\nIndex file '{index_path.name}' not found.")
    else:
        try:
            index_total_size = index_path.stat().st_size
            print(f"\nFound index file: {index_path.name}")
            with open(index_path, 'rb') as f: index_data = pickle.load(f)
            if isinstance(index_data, dict):
                 index_entry_count = len(index_data); print(f"Index contains {index_entry_count} entries.")
                 count = 0; print("  Sample Index Entries:")
                 for key, value in index_data.items():
                     path_val=value.get('path','N/A'); type_val=value.get('type','N/A'); stat_val=value.get('stat','N/A')
                     g=value.get('g','N/A'); l=value.get('l','N/A'); n=value.get('n','N/A')
                     print(f"    ID: {key} -> Path: {path_val}, Type: {type_val}, Stat: {stat_val}, G:{g}, L:{l}, N:{n}")
                     count += 1;
                     if count >= 5: break
            else: print("  Error: Index file content is not a dictionary."); index_entry_count = 0
        except Exception as e: print(f"  Error opening or reading index file '{index_path}': {e}"); index_entry_count = 0
        print(f"Total Index File Size: {index_total_size / 1024:.2f} KB")

    # --- 2. Анализ по Индексу ---
    print(f"\nAnalyzing tensors listed in index ({index_entry_count} entries)...")
    if index_entry_count > 0 and isinstance(index_data, dict):
        for tensor_id, index_entry in index_data.items():
            meta_tuple = None; tensor_type = "unknown"
            relative_meta_path = index_entry.get('path')
            if not relative_meta_path:
                 print(f"  WARN: Missing 'path' in index entry for ID: {tensor_id}"); error_count += 1; continue

            meta_file_path = DB_PATH / relative_meta_path

            if not meta_file_path.is_file():
                print(f"  WARN: Meta file referenced in index not found: {meta_file_path}")
                missing_meta_files += 1; continue # Эта запись индекса невалидна

            # Файл существует, увеличиваем счетчик и размер
            total_meta_files_in_index += 1
            try: total_meta_size += meta_file_path.stat().st_size
            except Exception: pass # Игнорируем ошибку получения размера

            # Загружаем КОРТЕЖ метаданных
            try:
                with open(meta_file_path, 'rb') as f: meta_tuple = pickle.load(f)
            except Exception as load_e:
                 print(f"  ERROR loading meta file {meta_file_path}: {load_e}"); error_count += 1; continue

            # Валидируем КОРТЕЖ
            if not validate_tensor_tuple(meta_tuple):
                 print(f"  WARN: Invalid metadata tuple structure in {meta_file_path.name}"); invalid_tuples += 1; continue

            # Анализируем КОРТЕЖ с помощью НОВЫХ геттеров
            try:
                type_code = get_type_code_from_meta(meta_tuple)
                tensor_type = REVERSE_DATA_TYPE_MAPPING.get(type_code, "unknown")

                # Считаем типы
                if tensor_type == "processor": processor_count += 1
                elif tensor_type == "knowledge": knowledge_count += 1
                elif tensor_type == "state": state_count += 1
                elif tensor_type == "converter": converter_count += 1
                else: other_count +=1

                # Проверяем блоб по флагу из кортежа
                if get_has_blob_flag_from_meta(meta_tuple) == 1:
                    tags = get_tags_list_from_meta(meta_tuple)
                    dtype_code = get_dtype_code_from_meta(meta_tuple)
                    coord_list = get_coord_list_from_meta(meta_tuple)

                    if not coord_list or not ANALYSIS_IMPORTS_OK: continue # Не можем построить путь

                    # Реконструируем объект координат ТОЛЬКО для пути
                    coord_obj = TensorCoordinate(*coord_list) # Не используем from_string
                    if not coord_obj: continue

                    # Определяем формат блоба
                    blob_format = 'pickle'
                    is_int8 = (dtype_code == DTYPE_MAPPING.get('int8'))
                    is_embed_or_lm = (TAG_COMP_EMBEDDING in tags) or (TAG_COMP_LM_HEAD in tags)
                    if is_embed_or_lm or is_int8: blob_format = 'npy'

                    # Проверяем файл блоба
                    blob_file = meta_file_path.parent / f"{tensor_id}.{blob_format}"
                    if blob_file.is_file():
                        try:
                            total_blob_size += blob_file.stat().st_size
                            if blob_format == 'npy': total_blob_files_npy += 1
                            else: total_blob_files_pickle += 1
                        except Exception as stat_e: print(f"  WARN: Cannot get size for {blob_file}: {stat_e}")

            except Exception as parse_e:
                print(f"  ERROR parsing metadata tuple from {meta_file.name}: {parse_e}"); error_count += 1;
                # Коррекция счетчиков типов при ошибке парсинга
                if tensor_type == "processor": processor_count = max(0, processor_count - 1)
                elif tensor_type == "knowledge": knowledge_count = max(0, knowledge_count - 1)
                # ... и т.д.

    # --- Печать итогов ---
    print(f"\n--- Analysis Summary (Based on Index) ---")
    print(f"Total Index Entries: {index_entry_count}")
    print(f"Meta Files Found (from index): {total_meta_files_in_index}")
    if missing_meta_files > 0: print(f"  WARNING: Missing meta files referenced in index: {missing_meta_files}")
    if invalid_tuples > 0: print(f"  WARNING: Invalid metadata tuples found: {invalid_tuples}")
    print(f"Total Metadata Size (of found files): {total_meta_size / 1024:.2f} KB")
    print(f"Total Blob Files (.npy): {total_blob_files_npy}")
    print(f"Total Blob Files (.blob): {total_blob_files_pickle}")
    total_blobs = total_blob_files_npy + total_blob_files_pickle
    print(f"Total Blob Files (sum): {total_blobs}")
    print(f"Total Blob Size: {total_blob_size / (1024*1024):.2f} MB")
    if total_blobs > 0: avg_blob_kb = (total_blob_size / total_blobs) / 1024; print(f"Average Blob Size: {avg_blob_kb:.2f} KB")
    print(f"Tensor Counts (based on 'type_code' in successfully parsed meta tuples):")
    print(f"  Knowledge: {knowledge_count}")
    print(f"  Processor: {processor_count}")
    print(f"  State: {state_count}")
    print(f"  Converter: {converter_count}")
    print(f"  Other/Unknown Type: {other_count}")
    print(f"  Errors during tuple processing: {error_count + invalid_tuples}") # Суммируем ошибки парсинга и невалидные кортежи

end_cell7_time = time.time()
print(f"--- Analysis Cell Finished in {end_cell7_time - start_cell7_time:.2f} seconds ---")


--- Running DB Analysis Cell (Index-First Analysis) ---
DEBUG: Importing from tensors version: 0.7.6
DEBUG: Imports for analysis successful.

--- Analyzing Veector DB Structure (Index-First Method) ---
Analyzing DB at: /content/data/db

Found index file: tensor_index.pkl
Index contains 398 entries.
  Sample Index Entries:
    ID: fb4ef375e208da470d6841a89c441ca29e016873a80c8519660c22cfa227be8d -> Path: g100/l-1/n1/fb4ef375e208da470d6841a89c441ca29e016873a80c8519660c22cfa227be8d.meta, Type: knowledge, Stat: active, G:100, L:-1, N:1
    ID: 3e405f866732b9da27a6bc1c1ff7b0977a66bc51b7e58a476f9f559855c46dc1 -> Path: g100/l0/n1/3e405f866732b9da27a6bc1c1ff7b0977a66bc51b7e58a476f9f559855c46dc1.meta, Type: knowledge, Stat: active, G:100, L:0, N:1
    ID: 7e35759fe8c401a67215f0914efcf0130a3523aec2106b85ccc604a9204ba17d -> Path: g100/l0/n1/7e35759fe8c401a67215f0914efcf0130a3523aec2106b85ccc604a9204ba17d.meta, Type: knowledge, Stat: active, G:100, L:0, N:1
    ID: 5edca5c7dcc96773e1d3801d80799891

In [16]:
# === Cell 8: Detailed Index File Analysis ===

import pickle
from pathlib import Path
import traceback
from collections import Counter

print(f"\n--- Running Detailed Index Analysis Cell ---")
start_cell8_time = time.time()

# --- Конфигурация ---
if 'DB_PATH' not in locals() or not isinstance(DB_PATH, Path):
     print("ERROR: DB_PATH not defined. Assuming './data/db/'")
     DB_PATH = Path("./data/db/")

INDEX_FILENAME = "tensor_index.pkl"
index_path = DB_PATH / INDEX_FILENAME

print(f"Attempting to load index file: {index_path}")

# --- Загрузка и Анализ ---
index_data = None
load_error = None
index_entry_count = 0
type_counts = Counter() # Счетчик для типов
loaded_ids = []

if not index_path.is_file():
    print(f"ERROR: Index file not found at {index_path}")
else:
    try:
        with open(index_path, 'rb') as f:
            # Пытаемся загрузить весь файл
            index_data = pickle.load(f)

        if isinstance(index_data, dict):
            index_entry_count = len(index_data)
            print(f"SUCCESS: Index file loaded successfully.")
            print(f"Total entries loaded by pickle: {index_entry_count}")

            # Анализируем загруженные записи
            print("\nAnalyzing loaded index entries...")
            for tensor_id, entry_data in index_data.items():
                if isinstance(entry_data, dict):
                    tensor_type = entry_data.get('type', 'unknown')
                    type_counts[tensor_type] += 1
                    loaded_ids.append(tensor_id)
                else:
                    print(f"  WARN: Invalid entry format for ID {tensor_id}: {type(entry_data)}")
                    type_counts['invalid_format'] += 1

        else:
            load_error = f"Loaded object is not a dictionary (Type: {type(index_data)})."
            print(f"ERROR: {load_error}")
            index_entry_count = 0 # Считаем, что ничего не загрузили корректно

    # Ловим конкретные ошибки UnpicklingError
    except pickle.UnpicklingError as upe:
        load_error = f"UnpicklingError: File corrupted or invalid format ({upe})."
        print(f"ERROR during pickle load: {load_error}")
        # Попытаться прочитать файл построчно или частично здесь сложно
        index_entry_count = 0
    except EOFError:
        load_error = "EOFError: File seems truncated or corrupted (unexpected end)."
        print(f"ERROR during pickle load: {load_error}")
        index_entry_count = 0
    except Exception as e:
        load_error = f"Unexpected error during pickle load: {e}"
        print(f"ERROR during pickle load: {load_error}")
        traceback.print_exc()
        index_entry_count = 0

# --- Вывод Результатов ---
print("\n--- Index Analysis Results ---")
if load_error:
    print(f"Index Loading Status: FAILED ({load_error})")
elif index_data is not None:
    print(f"Index Loading Status: SUCCESS")
    print(f"Number of Entries Loaded: {index_entry_count}")
    print("Counts by Tensor Type (from loaded index):")
    for tensor_type, count in type_counts.items():
        print(f"  - {tensor_type}: {count}")
    # Опционально: вывести все ID (может быть очень много)
    # print("\nLoaded Tensor IDs:")
    # for tensor_id in loaded_ids:
    #     print(f"  {tensor_id}")
else:
     print("Index Loading Status: File not found.")


end_cell8_time = time.time()
print(f"\n--- Index Analysis Cell Finished in {end_cell8_time - start_cell8_time:.2f} seconds ---")


--- Running Detailed Index Analysis Cell ---
Attempting to load index file: data/db/tensor_index.pkl
SUCCESS: Index file loaded successfully.
Total entries loaded by pickle: 398

Analyzing loaded index entries...

--- Index Analysis Results ---
Index Loading Status: SUCCESS
Number of Entries Loaded: 398
Counts by Tensor Type (from loaded index):
  - knowledge: 339
  - processor: 59

--- Index Analysis Cell Finished in 0.00 seconds ---


In [17]:
# Архивация и скачивание
import shutil
shutil.make_archive("model_DeepSeek-r1-distill-1.5b", "zip", "data")
zip_name = "model_DeepSeek-r1-distill-1.5b.zip"

In [ ]:
!ls -lh  # Посмотрите список файлов в текущей директории

In [18]:
# Выгрузка на Google Drive
drive.mount('/content/drive', force_remount=True)
destination_path = f"/content/drive/My Drive/models/"
shutil.copy(zip_name, destination_path)
print(f"🟢 [LOG] ✅ Архив загружен на Google Drive: {destination_path}")

Mounted at /content/drive
🟢 [LOG] ✅ Архив загружен на Google Drive: /content/drive/My Drive/models/


Saving processor tensors.
Do not forget save
tensor_index.pkl,
DeepSeek-R1-Distill-Qwen-1.5B_name_id_map.pkl,
DeepSeek-R1-Distill-Qwen-1.5B_proc_map.pkl

In [15]:
# Архивация и скачивание
import shutil
shutil.make_archive("g500", "zip", "data/db/g500")
zip_name = "g500.zip"